In [1]:
# General
import pandas as pd
import numpy as np
import polars as pl
# from pandarallel import pandarallel
# pandarallel.initialize()

from datetime import datetime, timedelta, date
import json
import ast
import glob
import os
import pickle
import re
from itertools import groupby
import time

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
# # Bigquery
# from google.oauth2 import service_account
# from google.cloud import bigquery
# from google.cloud import bigquery_storage

import warnings
warnings.filterwarnings('ignore')

import pyarrow.parquet as pq
import pyarrow.fs as fs
import dask.dataframe as dd
import vaex

pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', -1)  # or 199
# pd.set_option("display.max_rows", None) 

In [2]:
import plotly as py
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [3]:
# define function here
def f(x):
    try:
        return ast.literal_eval(str(x))   
    except:
        return []

In [4]:
parquet_files = ['Data_stroke.parquet']
df = vaex.open(parquet_files).to_pandas_df()
df.head()

visits    rowID  \
0  2       1000135   
1  1       1000246   
2  2       1000274   
3  2       1000311   
4  1       100039    

                                                                cid class  \
0  b472531d4caaecc893dc345f31ee8317ea459926dd37f0da1c50d2af2e8543aa  AMB    
1  1cf272b413de5fc4e1166e3b5aa35e146125efd0f33ad13a73f58368da638167  AMB    
2  239378b8bc40473f3f73235f6306c0ebb1d50af8aae434de8f75f0bac4fcd742  AMB    
3  b360f3bf1549170ef3d569c7f618f004bd2b78d210b8c7f80000ba75718cf01c  AMB    
4  4bad23f241ec47aeb6f333bbc87a7b4da1b6ded6969d21789ddb638be41ea985  AMB    

  visitDateTime hospCode             diagText  \
0 2022-08-16     10832    cerebral infarction   
1 2020-03-02     10685    None                  
2 2020-03-02     10753    None                  
3 2022-08-01     10832    old CVA, HT           
4 2021-01-14     10697    None                  

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       diagnoses  \
0  [{"icd10":"I639","icd10Name":"cerebral infarction;unspecified","diagType":"1","diagTypeName":""}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
1  [{"icd10":"I610","icd10Name":"Intracerebral haemorrhage in hemisphere, subcortical","diagType":"1","diagTypeName":"Principal Diagnosis"},{"icd10":"I10","icd10Name":"Essential (primary) hypertension","diagType":"2","diagTypeName":"Comorbidity (โรคอื่นที่เป็นร่วมด้วย)"},{"icd10":"E785","icd10Name":"Hyperlipidaemia, unspecified","diagType":"2","diagTypeName":"Comorbidity (โรคอื่นที่เป็นร่วมด้วย)"},{"icd10":"Z133","icd10Name":"Special screening examination for mental and behavioural disorders","diagType":"4","diagTypeName":"Other (โรคอื่น ๆ)"}]                                                                                                                              
2  [{"icd10":"I64","icd10Name":"Stroke,not specified as haemorrhage or infarction","diagType":"1","diagTypeName":"Principal Diagnosis"},{"icd10":"E149","icd10Name":"U-DM Without complications","diagType":"2","diagTypeName":"Comorbidity (โรคอื่นที่เป็นร่วมด้วย)"},{"icd10":"I10","icd10Name":"Essential (primary) hypertension","diagType":"2","diagTypeName":"Comorbidity (โรคอื่นที่เป็นร่วมด้วย)"},{"icd10":"I251","icd10Name":"Atherosclerotic heart disease","diagType":"2","diagTypeName":"Comorbidity (โรคอื่นที่เป็นร่วมด้วย)"},{"icd10":"E789","icd10Name":"Disorder of lipoprotein metabolism,unspecified","diagType":"2","diagTypeName":"Comorbidity (โรคอื่นที่เป็นร่วมด้วย)"}]   
3  [{"icd10":"I64","icd10Name":"CVA","diagType":"1","diagTypeName":""},{"icd10":"I10","icd10Name":"blood pressure high (see also hypertension)     ***** ยกเว้นผู้ป่วยตั้งครรภ์ คลอด หลังคลอด ******","diagType":"2","diagTypeName":""},{"icd10":"R42","icd10Name":"Dizziness","diagType":"2","diagTypeName":""}]                                                                                                       

In [5]:
df.shape

(111011, 76)

In [6]:
_col = ['visits','rowID','cid','vital_cc','vital_pe', 'visitDateTime',"icd10_list",'diagType_1','diagType_2','diagType_3','patientAge', "patientSexName", "smoking", "narcotic", "exercise",
 "alcohol","vital_bw","vital_waist", "vital_height","vital_bmi",'vital_hr','vital_pulse','vital_temperature','vital_rr',"vital_bps", "vital_bpd"]
data_col = df.loc[:, _col]

In [7]:
# optimized version of extract_labResults fucntion
# Using the .get() method to access dictionary values with a default value, instead of using indexing. This way, the code doesn't raise an error when the key is missing.
# Using the continue statement to skip to the next iteration in the for loops when the conditions are not met. This avoids having to indent the rest of the code inside the loops.

def extract_labResults(lab_list_dict):
    result = {} 
    for i, dictionary in enumerate(lab_list_dict): # enumerate เพิ่มตัวเลขจำนวนการลูป
        lab_head_data = dictionary.get('labHeadData', {}) # ไปเอา value ของ key labHeadData มา ถ้าไม่มีค่าก็เป็น {}
        if lab_head_data.get('confirmReport') != 'Y':
            continue # ถ้าไม่ Y ก็ไม่ต้องทำต่อ (ข้ามไปเลย) ไปทำใหม่
        result[i] = {
            'departmentName': lab_head_data.get('departmentName'), 
            'spcltyName': lab_head_data.get('spcltyName'),
            'labReportData': {}
        }
        lab_list = dictionary.get('labReportData', []) # ไปเอา value ของ key labReportData มา ถ้าไม่มีค่าก็เป็น {}
        for d in lab_list:
            if d.get('confirm') != 'Y':
                continue
            result[i]['labReportData'][d['labItemsCode']] = {
                'labItemsNameRef': d['labItemsNameRef'],
                'labOrderResult': d['labOrderResult'],
                'labItemsNormalValueRef': d['labItemsNormalValueRef'],
                'labItemsUnit': d['labItemsUnit']
            }
    return result

In [8]:
def clean_labResults(lab_result_2):
    '''
    To clean these 3 -> labOrderResult, labItemsNormalValueRef, labItemsUnit
    
    [labOrderResult] # เอาเเค่ตัวเลข (ค่าเเลป) ไม่เอาช่วง
    Mostly labOrderResult is filled with number or string pattern but some of it has like "0 - 0 Unit" pattern
    which it may be the labItemsNormalValueRef and labItemsUnit(Obviously not supposed to be in labOrderResult).
    So, in "0 - 0 Unit" case, we can move it to both of them if it is empthy or "-".
    
    Moreover, In labOrderResult, we'll only keep the number value (e.g.63.19 (Stage 2) -> use 63.19).
    
    [labItemsNormalValueRef] # ไม่เอา unit 
    "0 - 0" , "0 - 0 Unit" and string should be a pattern of labItemsNormalValueRef 
    but sometimes, labItemsNormalValueRef is exactly like labItemsUnit.
    So, if we can't extract it by using the regex labItemsNormalValueRef pattern and they're exacly alike.
    then we'll remove the labItemsNormalValueRef and leave the labItemsUnit there.
    '''
    
    number_unit_regex = r'^(-?\d+(?:[\.\,]\d+)?\s*-\s*-?\d+(?:[\.\,]\d+)?|[>=<]*\s*-?\d+(?:[\.\,]\d+)?)(.*)'

    for labHeadDataKey, labHeadData in lab_result_2.items():
        for labItemsCode, lab_data in labHeadData['labReportData'].items():
            # Extract number and unit from labOrderResult
            lab_order_result = lab_data['labOrderResult'].strip()
            lab_order_result_match = re.match(number_unit_regex, lab_order_result)
            if lab_order_result_match:
                try:
                    lab_data['labOrderResult'] = lab_order_result_match.group(1).strip()
                    if "stage" not in lab_order_result_match.group(2).lower():
                        lab_data['labItemsUnit'] = lab_order_result_match.group(2).strip()
                except IndexError:
                    pass

            # Extract number and unit from labItemsNormalValueRef
            lab_items_normal_value_ref = lab_data['labItemsNormalValueRef'].strip()
            lab_items_normal_value_ref_match = re.match(number_unit_regex, lab_items_normal_value_ref)
            if lab_items_normal_value_ref_match:
                try:
                    lab_data['labItemsNormalValueRef'] = lab_items_normal_value_ref_match.group(1)
                    if (lab_data['labItemsUnit'] == '' or lab_data['labItemsUnit'] == '-') and "stage" not in lab_items_normal_value_ref_match.group(2).lower():
                        lab_data['labItemsUnit'] = lab_items_normal_value_ref_match.group(2)
                except IndexError:
                    pass
            elif lab_data['labItemsNormalValueRef'].strip().lower() == lab_data['labItemsUnit'].strip().lower():
                lab_data['labItemsNormalValueRef'] = ''

    return lab_result_2


In [9]:
def extract_labResults_data(data):
    df = data.copy()
    df['lab_result_2'] = df['labResults'].apply(json.loads).apply(extract_labResults)
    return df

def clean_lab_result_data(data):
    df = data.copy()
    df['lab_result_3'] = df['lab_result_2'].apply(clean_labResults)
    return df

def extract_labResults3(df):
    lab_result = df.get("labResults")
    if lab_result and lab_result.get(df["labHeadData"]):
        return lab_result[df["labHeadData"]]
    return None

def extract_labReportData(df):
    if pd.isna(df['labReportData']) != True:
        for i in range(len(list(df['labReportData'].items()))):
            if list(df['labReportData'].items())[i][0] == df['labItemCode']:
                return list(df['labReportData'].items())[i][1]
    else:
        return df['labReportData']
    
def clean(df):
    df2 = extract_labResults_data(df)
    df2 = clean_lab_result_data(df2)
    df2.drop(['labResults','lab_result_2'], axis=1, inplace=True)
    df2.rename(columns = {'lab_result_3' : 'labResults'},inplace=True)
    return df2

def clean2(df2):
    ex = df2.explode('labResults')
    ex.rename(columns = {'labResults' : 'labHeadData'},inplace=True)
    df_merge = ex.merge(df2, how = 'left' , on=['visits','rowID','cid','visitDateTime'])
    df_merge['labHeadData2'] = df_merge.apply(extract_labResults3, axis=1)
    df_merge['labHeadData2'].fillna(value = np.NaN, inplace=True)
    df_merge['labHeadData2'] = df_merge['labHeadData2'].apply(lambda x: {} if pd.isna(x) else x)
    df_concat = pd.concat([df_merge[['visits','rowID', 'cid', 'visitDateTime','labHeadData']], pd.json_normalize(df_merge['labHeadData2'], max_level = 0)], axis=1)
    df_concat_ex = df_concat.explode('labReportData').reset_index(drop=True)
    df_concat_ex.rename(columns = {'labReportData':'labItemCode'}, inplace=True)
    df_merge2 = df_concat_ex.merge(df_concat, how = 'left' , on=['visits','rowID','cid','visitDateTime','labHeadData','departmentName','spcltyName'])
    df_merge2['labReportData2'] = df_merge2.apply(extract_labReportData, axis=1)
    df_merge2.drop(['labReportData'], axis=1,inplace=True)
    df_merge2.rename(columns={'labReportData2':'labReportData'}, inplace=True)
    df_concat2 = pd.concat([df_merge2[['visits','rowID','cid','visitDateTime','labHeadData','departmentName','spcltyName','labItemCode']], pd.json_normalize(df_merge2['labReportData'])], axis=1)
    df_concat2 = df_concat2.reindex(columns = ['visits','rowID','cid','visitDateTime','labHeadData','departmentName',
       'spcltyName','labItemCode','labItemsNameRef','labOrderResult','labItemsNormalValueRef','labItemsUnit'])
    return df_concat2

In [10]:
%%time
df2 = clean(df)
df_concat2 = clean2(df2)
df_concat2

Wall time: 1min 10s


,visits,rowID,cid,visitDateTime,labHeadData,departmentName,spcltyName,labItemCode,labItemsNameRef,labOrderResult,labItemsNormalValueRef,labItemsUnit
0,2,1000135,b472531d4caaecc893dc345f31ee8317ea459926dd37f0da1c50d2af2e8543aa,2022-08-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1000246,1cf272b413de5fc4e1166e3b5aa35e146125efd0f33ad13a73f58368da638167,2020-03-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1000274,239378b8bc40473f3f73235f6306c0ebb1d50af8aae434de8f75f0bac4fcd742,2020-03-02,0,งานหน้าห้องตรวจโรคทั่วไป 1,อายุรกรรม GP,1001,GLU,231,74.00 - 106.00,
3,2,1000311,b360f3bf1549170ef3d569c7f618f004bd2b78d210b8c7f80000ba75718cf01c,2022-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,100039,4bad23f241ec47aeb6f333bbc87a7b4da1b6ded6969d21789ddb638be41ea985,2021-01-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
651096,1,999629,9638a995f2836643b505771c00668f0d6951b1084f10b5735051dd85ffc51534,2020-03-02,0,ฝ่ายเวชระเบียน,ชลอไต,5287,Tear drop,,,
651097,1,999629,9638a995f2836643b505771c00668f0d6951b1084f10b5735051dd85ffc51534,2020-03-02,0,ฝ่ายเวชระเบียน,ชลอไต,5288,Stomatocyte,,,
651098,1,999629,9638a995f2836643b505771c00668f0d6951b1084f10b5735051dd85ffc51534,2020-03-02,0,ฝ่ายเวชระเบียน,ชลอไต,5334,eGFR,48.02,,
651099,5,999834,507b2eed38b70f8f1f122b11c869b60713c95a4e4dbfaddf572bcbe18f00320e,2022-08-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
#before
df_concat2['labItemsNameRef'].value_counts()[0:50]

Creatinine             17524
BUN                    12639
MCH                    10940
MCV                    10940
MCHC                   10596
RBC                    9725 
Monocyte               9359 
Lymphocyte             9359 
Triglyceride           9232 
Potassium              9044 
Eosinophil             9015 
Basophil               9015 
eGFR                   8659 
Sodium                 8224 
Fasting glucose        8181 
Chloride               8007 
e-GFR (CKD-EPI)        7867 
e-GFR (MDRD)           7867 
Cholesterol            7781 
Neutrophil             7394 
WBC                    7304 
INR                    7080 
LDL-chelesterol        5946 
HGB                    5766 
PT                     5705 
HCT                    5396 
Bicarbonate            4697 
HbA1c                  4558 
SGPT (ALT)             4223 
absolute neutrophil    4084 
SGOT (AST)             4064 
Platelet (count)       4023 
Platelet (smear)       4023 
Hb                     3470 
RDW-CV        

In [12]:
# replace_values = open("replace_Value.txt", "r")
# df_concat2.replace({"labItemsNameRef": replace_values}, inplace=True)

In [13]:
%%time
# replace name lab 
replace_values = json.loads(open('replace_values_lab.json', 'r', encoding="utf8").read()) # utf8 : basic ตัวอักษร
df_concat2.replace({'labItemsNameRef': replace_values}, inplace=True)

Wall time: 5min 30s


In [14]:
# after
df_concat2['labItemsNameRef'].value_counts()[:22] # look lab top 50 

Creatinine         22544
FBS                21228
RBC                16231
WBC                16074
LDL-Cholesterol    14580
Potassium          14578
BUN                13470
HCT                13172
CO2                13147
Sodium             12513
PLT                12279
MCV                11948
MCH                11947
Triglyceride       11752
MCHC               11497
Basophil           10938
Neutrophil         10928
Lymphocytes        10920
Monocytes          10920
Eosinophil         10920
Chloride           10315
Cholesterol        9883 
Name: labItemsNameRef, dtype: int64

In [15]:
asas

NameError: name 'asas' is not defined

## Check labItemsUnit 

In [ ]:
# Creatinine
# Cholesterol
# LDL-Cholesterol
# FBS
# Triglyceride
# Potassium
# HCT

In [16]:
selected_lab_df = df_concat2.loc[:, ["rowID", "cid", "labItemsNameRef","labItemsNormalValueRef", "labOrderResult","labItemsUnit"]]

In [17]:
unique_labItemsNameRef = selected_lab_df.groupby(['labItemsNameRef','labItemsNormalValueRef','labItemsUnit']).agg(['nunique']).reset_index(drop=False)
# unique_labItemsNameRef.loc[unique_labItemsNameRef['labItemsNameRef']=="HCT"]

In [18]:
unique_labItemsNameRef.iloc[:, 0:4]

,labItemsNameRef,labItemsNormalValueRef,labItemsUnit,rowID
,,,,nunique
0,,,,42
1,% A,%,-,1
2,% A2,2.0 - 3.5,%,1
3,% E,%,-,1
4,% F,<1.0,%,1
...,...,...,...,...
3728,ครั้งที่ 1,,-,1
3729,ตำแหน่งรอยโรค,,,1
3730,วิธีที่ 1,,COI,12


In [ ]:
df = selected_lab_df.loc[selected_lab_df['labItemsNameRef'] == 'HCT']
fig1 = px.histogram(data_frame =df,x='labOrderResult' )
                         
fig1.show()

In [ ]:
unique_labItemsNameRef.loc[unique_labItemsNameRef['labItemsNameRef']=="HCT"].value_counts()

In [ ]:
selected_lab_df.loc[selected_lab_df['labItemsNameRef'] == 'HCT'].value_counts()

In [ ]:
selected_lab_df.loc[(selected_lab_df['labItemsNameRef'] == 'HCT'), 'labItemsUnit'].value_counts()

In [ ]:
selected_lab_df.loc[(selected_lab_df['labItemsNameRef'] == 'HCT'), 'labItemsNormalValueRef'].value_counts()

In [ ]:
selected_lab_df.loc[selected_lab_df['labItemsNameRef'] == 'HCT'].shape[0]

In [19]:
selected_lab_df.loc[(selected_lab_df['labItemsNameRef'] == 'HCT'), 'labOrderResult'].unique()

array(['37.4', '27.6', '38', '26.9', '29.9', '47', '29.4', '35', '42.9',
       '36.9', '37.6', '40', '30', '30.4', '41', '41.1', '41.7', '26',
       '25', '43', '13.5', '14.8', '32.0', '37', '37.2', '40.9', '39',
       '23', '42', '36.3', '38.7', '45.3', '36.0', '42.7', '31', '42.0',
       '32', '29.6', '42.5', '57', '36.7', '46.2', '39.0', '35.8', '39.3',
       '49', '44', '38.8', '41.2', '47.7', '33.5', '32.9', '33.3', '40.0',
       '43.4', '43.5', '33.6', '44.5', '44.2', '36', '44.1', '42.2',
       '31.6', '42.3', '44.7', '41.3', '48', '36.2', '45.7', '28.9',
       '43.6', '45', '44.3', '29.8', '38.2', '40.4', '37.5', '45.0',
       '36.5', '47.1', '44.4', '34.5', '48.3', '46', '28.5', '52.3',
       '26.5', '40.5', '41.0', '35.6', '45.4', '43.8', '48.7', '43.3',
       '44.6', '42.4', '42.8', '40.6', '47.8', '39.1', '41.9', '38.0',
       '40.7', '45.6', '45.1', '49.7', '34', '52.1', '42.1', '36.1',
       '38.3', '52.5', '37.7', '44.9', '34.0', '32.3', '39.8', '33.9',
    

In [32]:
selected_lab_df.loc[(selected_lab_df['labItemsNameRef'] == 'HCT') & (selected_lab_df['labOrderResult'] == 'repeat 48.0'),
                    ['labOrderResult','labItemsNormalValueRef','labItemsUnit']]

,labOrderResult,labItemsNormalValueRef,labItemsUnit
257668,repeat 48.0,37.0 - 54.0,%


In [38]:
selected_lab_df['labOrderResult'] = selected_lab_df['labOrderResult'].apply(lambda x: str(x).replace('repeat' , '').strip())

In [35]:
selected_lab_df.loc[(selected_lab_df['labItemsNameRef'] == 'HCT') & (selected_lab_df['labOrderResult'] == 'repeat 48.0'),
                    ['labOrderResult','labItemsNormalValueRef','labItemsUnit']]

,labOrderResult,labItemsNormalValueRef,labItemsUnit


In [39]:
selected_lab_df.loc[(selected_lab_df['labItemsNameRef'] == 'HCT'), 'labOrderResult'].unique()

array(['37.4', '27.6', '38', '26.9', '29.9', '47', '29.4', '35', '42.9',
       '36.9', '37.6', '40', '30', '30.4', '41', '41.1', '41.7', '26',
       '25', '43', '13.5', '14.8', '32.0', '37', '37.2', '40.9', '39',
       '23', '42', '36.3', '38.7', '45.3', '36.0', '42.7', '31', '42.0',
       '32', '29.6', '42.5', '57', '36.7', '46.2', '39.0', '35.8', '39.3',
       '49', '44', '38.8', '41.2', '47.7', '33.5', '32.9', '33.3', '40.0',
       '43.4', '43.5', '33.6', '44.5', '44.2', '36', '44.1', '42.2',
       '31.6', '42.3', '44.7', '41.3', '48', '36.2', '45.7', '28.9',
       '43.6', '45', '44.3', '29.8', '38.2', '40.4', '37.5', '45.0',
       '36.5', '47.1', '44.4', '34.5', '48.3', '46', '28.5', '52.3',
       '26.5', '40.5', '41.0', '35.6', '45.4', '43.8', '48.7', '43.3',
       '44.6', '42.4', '42.8', '40.6', '47.8', '39.1', '41.9', '38.0',
       '40.7', '45.6', '45.1', '49.7', '34', '52.1', '42.1', '36.1',
       '38.3', '52.5', '37.7', '44.9', '34.0', '32.3', '39.8', '33.9',
    

In [ ]:
# # ตัดค่าที่ไม่ใช่ตัวเลข
# selected_lab_df.loc[selected_lab_df['labItemsNameRef']=='HCT'] = selected_lab_df.drop( index=selected_lab_df.loc[(selected_lab_df['labItemsNameRef']=='HCT') &
#                     (selected_lab_df['labOrderResult'].str.replace('.','').str.isnumeric() == False)].index    )

In [ ]:
# selected_lab_df.loc[selected_lab_df['labItemsNameRef'] == 'HCT','labOrderResult'].astype(float)

In [ ]:
df = selected_lab_df.loc[selected_lab_df['labItemsNameRef'] == 'LDL-Cholesterol']
fig1 = px.histogram(data_frame =df,x='labOrderResult' )
                         
fig1.show()

In [ ]:
selected_lab_df.loc[(selected_lab_df['labItemsNameRef'] == 'LDL-Cholesterol'), 'labItemsUnit'].value_counts()

In [20]:
selected_lab_df.loc[(selected_lab_df['labItemsNameRef'] == 'LDL-Cholesterol'), 'labOrderResult'].unique()

array(['85', '67', '71', '51', '204', '48', '40', '82', '174', '165',
       '60', '147', '169', '98', '88', '80', '52', '163', '94', '145',
       '83', '75', '44', '142', '56', '130', '63', '73', '127', '134',
       '64', '117', '97', '122', '92', '177', '101', '104', '103', '95',
       '140', '61', '93', '65', '99', '62', '84', '53', '86', '36', '102',
       '87', '120', '123', '72', '55', '153', '91', '81', '111', '189',
       '57', '79', '58', '106', '78', '77', '107', '90', '70', '100',
       '68', '29', '69', '190', '109', '89', '112', '124', '108', '172',
       '146', '121', '50', '182', '76', '139', '150', '126', '143', '74',
       '115', '154', '42', '37', '207', '194', '66', '128', '193', '110',
       '133', '54', '162', '118', '13', '164', '46', '173', '47', '119',
       '187', '149', '157', '125', '160', '137', '144', '105', '96', '59',
       '43', '49', '132', '39', '113', '34', '223', '188', '141', '114',
       '136', '158', '151', '205', '159', '33', '30', '1

In [31]:
selected_lab_df.loc[(selected_lab_df['labItemsNameRef'] == 'LDL-Cholesterol') & (selected_lab_df['labOrderResult'] == 'LDL-direct=101'),
                    ['labOrderResult','labItemsNormalValueRef','labItemsUnit']]

,labOrderResult,labItemsNormalValueRef,labItemsUnit
58871,LDL-direct=101,<100,mg/dL


In [ ]:
selected_lab_df.loc[(selected_lab_df['labItemsNameRef'] == 'LDL-Cholesterol'), 'labItemsNormalValueRef'].value_counts()

In [ ]:
selected_lab_df.loc[selected_lab_df['labItemsNameRef'] == 'LDL-Cholesterol'].shape[0]

In [ ]:
# selected_lab_df.loc[selected_lab_df['labItemsNameRef']=='LDL-Cholesterol'] = selected_lab_df.drop( index=selected_lab_df.loc[(selected_lab_df['labItemsNameRef']=='LDL-Cholesterol') &
#                     (selected_lab_df['labOrderResult'].str.replace('.','').str.isnumeric() == False)].index    )

In [ ]:
selected_lab_df.loc[selected_lab_df['labItemsNameRef'] == 'LDL-Cholesterol'].shape[0]

In [ ]:
df = selected_lab_df.loc[selected_lab_df['labItemsNameRef'] == 'FBS']
fig1 = px.histogram(data_frame =df,x='labOrderResult' )
                         
fig1.show()

In [ ]:
selected_lab_df.loc[(selected_lab_df['labItemsNameRef'] == 'FBS'), 'labItemsUnit'].value_counts()

In [54]:
selected_lab_df.loc[(selected_lab_df['labItemsNameRef'] == 'FBS'), 'labItemsNormalValueRef' ].value_counts()

74 - 109          8184
74 - 106          3482
70 - 110          2978
70 - 115          1966
70-110            1309
70.0 - 110.0      1008
Negative          543 
70 - 126          529 
70.00 - 110.00    334 
74.00 - 106.00    257 
70 -110           247 
60-110            148 
95                88  
Negative -        41  
70-105            41  
70-100            34  
74-106            19  
mg/dL             10  
79 - 115          5   
                  3   
120-140           2   
Name: labItemsNormalValueRef, dtype: int64

In [ ]:
selected_lab_df.loc[selected_lab_df['labItemsNameRef'] == 'FBS'].shape[0]

In [ ]:
# def clean_FBS(x):
#     x = x.replace(' ','')
#     x = x.replace(',',)
#     if re.search(r'\d+.*\d+',x):
#         return float(re.search(r'\d+',x).group())
#     elif re.search(r'>|<.*\d+',x):
#         return float(re.search(r'\d+',x).group())
#     else:
#         return float(x)

In [ ]:
# selected_lab_df.loc[(selected_lab_df['labItemsNameRef'] == 'FBS'), 'labOrderResult'] = selected_lab_df.loc[(selected_lab_df['labItemsNameRef'] == 'FBS'), 'labOrderResult'].apply(clean_FBS)

In [21]:
selected_lab_df.loc[(selected_lab_df['labItemsNameRef'] == 'FBS'), 'labOrderResult'].unique()

array(['95', '96', '87', '144', '115', '158', '169', '139', '127', '128',
       '131', '103', '152', '155', '108', '179', '117', '136', '126',
       '98', '104', '119', '97', '146', '92', '137', '101', '106',
       'Negative', '110', '233', '262', '251', '145', '154', '60', '91',
       '383', '99', '84', '124', '140', '56', '458', '222', '102', '122',
       '114', '210', '4', '356', '188', '162', '132', '208', '100', '82',
       '79', '86', '94', '112', '134', '109', '125', '190', '116', '113',
       '215', '105', '157', '323', '88', '151', '200', '89', '156', '135',
       '191', '182', '143', '93', '120', '318', '254', '78', '123', '160',
       '148', '167', '142', '121', '90', '227', '228', '138', '80', '111',
       '180', '174', '294', '171', '83', '164', '85', '213', '330', '589',
       '346', '284', '129', '440', '184', '166', '118', '277', '107',
       '133', '141', '150', '130', '159', '198', '185', '149', '178',
       '147', '77', '194', '183', '76', '221', '189', 

In [ ]:
# selected_lab_df.loc[selected_lab_df['labItemsNameRef']=='FBS'] = selected_lab_df.drop( index=selected_lab_df.loc[(selected_lab_df['labItemsNameRef']=='FBS') &
#                     (selected_lab_df['labOrderResult'].str.replace('.','').str.isnumeric() == False)].index    )

In [ ]:
df = selected_lab_df.loc[selected_lab_df['labItemsNameRef'] == 'Creatinine']
fig1 = px.histogram(data_frame =df,x='labOrderResult' )
                         
fig1.show()

In [ ]:
selected_lab_df.loc[(selected_lab_df['labItemsNameRef'] == 'Creatinine'), 'labItemsUnit'].value_counts()

In [ ]:
selected_lab_df.loc[(selected_lab_df['labItemsNameRef'] == 'Creatinine'), 'labItemsNormalValueRef'].value_counts()

In [ ]:
selected_lab_df.loc[selected_lab_df['labItemsNameRef'] == 'Creatinine'].shape[0]

In [22]:
selected_lab_df.loc[(selected_lab_df['labItemsNameRef'] == 'Creatinine'), 'labOrderResult'].unique()

array(['0.78', '0.57', '0.73', '0.99', '1.52', '1.56', '1.71', '2.80',
       '2.02', '6.56', '0.52', '0.45', '0.93', '0.61', '0.96', '0.60',
       '1.40', '0.92', '0.64', '1.43', '1.47', '1.08', '1.24', '1.17',
       '1', '1.12', '0.1', '1.36', '1.05', '1.19', '1.38', '0.85', '0.86',
       '4.11', '1.81', '10.88', '11.35', '0.83', '1.01', '0.8', '2.18',
       '0.90', '0.9', '1.64', '1.21', '0.81', '1.59', '1.1', '0.84',
       '1.0', '1.54', '0.67', '0.72', '0.97', '1.11', '4.00', '1.42',
       '0.89', '0.95', '0.70', '3.29', '0.76', '1.07', '0.65', '0.77',
       '1.10', '1.31', '1.35', '1.25', '1.50', '1.41', '1.72', '1.51',
       '0.74', '1.04', '0.58', '0.69', '0.56', '0.80', '2.62', '1.82',
       '1.20', '1.32', '1.28', '1.63', '1.85', '1.49', '1.99', '0.91',
       '1.02', '0.75', '1.62', '2.08', '1.13', '1.60', '0.87', '1.55',
       '0.62', '1.06', '0.46', '11.68', '1.00', '0.47', '0.66', '2.91',
       '0.79', '0.82', '1.98', '1.03', '0.53', '0.94', '0.88', '1.09',
   

In [ ]:
# selected_lab_df.loc[selected_lab_df['labItemsNameRef']=='Creatinine'] = selected_lab_df.drop( index=selected_lab_df.loc[(selected_lab_df['labItemsNameRef']=='Creatinine') &
#                     (selected_lab_df['labOrderResult'].str.replace('.','').str.isnumeric() == False)].index    )

In [ ]:
df = selected_lab_df.loc[selected_lab_df['labItemsNameRef'] == 'Potassium']
fig1 = px.histogram(data_frame =df,x='labOrderResult' )
                         
fig1.show()

In [ ]:
selected_lab_df.loc[(selected_lab_df['labItemsNameRef'] == 'Potassium'), 'labItemsUnit'].value_counts()

In [ ]:
selected_lab_df.loc[(selected_lab_df['labItemsNameRef'] == 'Potassium'), 'labItemsNormalValueRef'].value_counts()

In [ ]:
selected_lab_df.loc[selected_lab_df['labItemsNameRef'] == 'Potassium'].shape[0]

In [ ]:
# selected_lab_df.loc[selected_lab_df['labItemsNameRef']=='Potassium'] = selected_lab_df.drop( index=selected_lab_df.loc[(selected_lab_df['labItemsNameRef']=='Potassium') &
#                     (selected_lab_df['labOrderResult'].str.replace('.','').str.isnumeric() == False)].index    )

In [ ]:
selected_lab_df.loc[selected_lab_df['labItemsNameRef'] == 'Potassium'].shape[0]

In [23]:
selected_lab_df.loc[(selected_lab_df['labItemsNameRef'] == 'Potassium'), 'labOrderResult'].unique()

array(['4.12', '2.89', '3.7', '4.46', '5.04', '3.15', '3.78', '3.01',
       '4.2', '3.75', '4.6', '4.0', '3.2', '3.6', '5.50', '4.16', '4.66',
       '4.62', '3.0', '4.22', '3.40', '2.75', '4.88', '3.69', '4.28',
       '4.06', '3.9', '4.14', '3.86', '3.53', '3.99', '3.57', '3.60',
       '4.17', '-', '4.1', '3.4', '4.21', '3.8', '3.59', '3.03', '3.70',
       '4.41', '4.15', '4.26', '4.55', '4.49', '4.5', '3.47', '4.71',
       '3.5', '4.19', '3.90', '4.11', '4.50', '3.63', '3.77', '3.37',
       '4.13', '3.28', '3.93', '3.58', '3.82', '3.64', '3.52', '4.61',
       '3.67', '3.43', '3.12', '2.85', '5.0', '3.85', '3.66', '3.97',
       '3.80', '3.83', '3.96', '2.9', '5.7', '4.3', '3.68', '3.3', '4.04',
       '3.22', '3.50', '4.31', '3.92', '3.91', '3.31', '4.30', '4.53',
       '4.33', '3.49', '3.55', '3.73', '3.35', '3.42', '2.26', '4.8',
       '3.94', '3.05', '3.81', '4.29', '4.08', '3.98', '3.79', '4.9',
       '3.38', '4.4', '4.7', '3.54', '4.84', '3.74', '3.11', '3.19',
       

In [ ]:
df = selected_lab_df.loc[selected_lab_df['labItemsNameRef'] == 'Triglyceride']
fig1 = px.histogram(data_frame =df,x='labOrderResult' )
                         
fig1.show()

In [64]:
selected_lab_df.loc[(selected_lab_df['labItemsNameRef'] == 'Triglyceride'), 'labItemsUnit'].value_counts()

 mg/dL       6038
             2976
 mg/dl       2517
 -           205 
mg/dl        9   
mg/dL        3   
Lipemic1+    1   
mg%          1   
repeat       1   
 mg%         1   
Name: labItemsUnit, dtype: int64

In [65]:
selected_lab_df.loc[(selected_lab_df['labItemsNameRef'] == 'Triglyceride'), 'labItemsNormalValueRef'].value_counts()

< 200             4246
45 - 150          1321
< 150             1290
0 - 150           1189
50 - 200          569 
40 - 160          416 
<= 204            366 
50-200            357 
0 - 149           326 
0.00 - 149.00     318 
35 - 135          266 
35 - 160          228 
200               199 
60.00 - 150.00    168 
0 - 180           138 
140-160           126 
30 - 150          123 
35-160            59  
40-150            21  
0-150             16  
0-180             5   
30-170            3   
<150              2   
Name: labItemsNormalValueRef, dtype: int64

In [ ]:
selected_lab_df.loc[selected_lab_df['labItemsNameRef'] == 'Triglyceride'].shape[0]

In [24]:
selected_lab_df.loc[(selected_lab_df['labItemsNameRef'] == 'Triglyceride'), 'labOrderResult'].unique()

array(['50', '80', '205', '203', '98', '111', '94', '87', '107', '161',
       '225', '83', '86', '73', '118', '121', '148', '147', '227', '169',
       '252', '74', '119', '289', '259', '170', '164', '63', '166', '186',
       '79', '221', '178', '207', '159', '89', '334', '56', '106', '153',
       '69', '102', '105', '55', '104', '126', '144', '143', '81', '72',
       '141', '90', '84', '282', '132', '103', '123', '92', '117', '85',
       '150', '70', '51', '58', '191', '99', '115', '116', '174', '263',
       '167', '184', '142', '112', '67', '78', '82', '248', '76', '171',
       '101', '193', '96', '61', '127', '145', '146', '134', '204', '65',
       '62', '206', '45', '49', '77', '75', '162', '229', '218', '149',
       '129', '138', '236', '196', '91', '137', '283', '133', '272', '71',
       '181', '124', '95', '130', '43', '173', '393', '139', '234', '136',
       '152', '177', '156', '189', '113', '175', '155', '684', '97',
       '125', '722', '68', '220', '120', '215', 

In [ ]:
# selected_lab_df.loc[selected_lab_df['labItemsNameRef']=='Triglyceride'] = selected_lab_df.drop( index=selected_lab_df.loc[(selected_lab_df['labItemsNameRef']=='Triglyceride') &
#                     (selected_lab_df['labOrderResult'].str.replace('.','').str.isnumeric() == False)].index    )

In [ ]:
df = selected_lab_df.loc[selected_lab_df['labItemsNameRef'] == 'Cholesterol']
fig1 = px.histogram(data_frame =df,x='labOrderResult' )
                         
fig1.show()

In [ ]:
selected_lab_df.loc[(selected_lab_df['labItemsNameRef'] == 'Cholesterol'), 'labItemsUnit'].value_counts()

In [ ]:
selected_lab_df.loc[(selected_lab_df['labItemsNameRef'] == 'Cholesterol'), 'labItemsNormalValueRef'].value_counts()

In [ ]:
selected_lab_df.loc[selected_lab_df['labItemsNameRef'] == 'Cholesterol'].shape[0]

In [25]:
selected_lab_df.loc[(selected_lab_df['labItemsNameRef'] == 'Cholesterol'), 'labOrderResult'].unique()

array(['147', '116', '146', '295', '108', '107', '113', '130', '250',
       '255', '112', '223', '166', '145', '142', '111', '264', '162',
       '241', '136', '102', '240', '152', '128', '183', '171', '105',
       '182', '139', '276', '193', '197', '172', '159', '158', '186',
       '131', '155', '129', '125', '218', '119', '123', '144', '181',
       '194', '167', '164', '135', '165', '247', '137', '169', '176',
       '140', '126', '151', '132', '154', '173', '170', '175', '138',
       '150', '157', '109', '163', '149', '177', '122', '148', '246',
       '196', '106', '260', '121', '232', '207', '189', '133', '178',
       '88', '199', '161', '141', '191', '219', '120', '124', '221',
       '160', '192', '236', '243', '200', '204', '76', '134', '239',
       '184', '117', '188', '168', '185', '90', '174', '235', '245',
       '115', '210', '212', '238', '201', '202', '114', '103', '206',
       '143', '127', '94', '214', '91', '275', '270', '99', '118', '180',
       '190', '226'

In [ ]:
# selected_lab_df.loc[selected_lab_df['labItemsNameRef']=='Cholesterol'] = selected_lab_df.drop( index=selected_lab_df.loc[(selected_lab_df['labItemsNameRef']=='Cholesterol') &
#                     (selected_lab_df['labOrderResult'].str.replace('.','').str.isnumeric() == False)].index    )

In [ ]:
###############################

In [ ]:
# df = selected_lab_df.loc[selected_lab_df['labItemsNameRef'] == 'RBC']
# fig1 = px.histogram(data_frame =df,x='labOrderResult' )
                         
# fig1.show()

In [ ]:
# selected_lab_df.loc[(selected_lab_df['labItemsNameRef'] == 'RBC'), 'labItemsUnit'].value_counts()

In [ ]:
# selected_lab_df.loc[(selected_lab_df['labItemsNameRef'] == 'RBC'), 'labOrderResult'].value_counts()

In [ ]:
# selected_lab_df.loc[(selected_lab_df['labItemsNameRef']=='RBC') & (selected_lab_df['labOrderResult'].str.replace('.','').str.isnumeric() == False),'labOrderResult'].value_counts()

In [ ]:
# selected_lab_df.loc[selected_lab_df['labItemsNameRef'] == 'RBC'].shape[0]

In [ ]:
# selected_lab_df.loc[selected_lab_df['labItemsNameRef']=='RBC'] = selected_lab_df.drop( index=selected_lab_df.loc[(selected_lab_df['labItemsNameRef']=='RBC') &
#                     (selected_lab_df['labOrderResult'].str.replace('.','').str.isnumeric() == False)].index    )

In [ ]:
# df = selected_lab_df.loc[selected_lab_df['labItemsNameRef'] == 'WBC']
# fig1 = px.histogram(data_frame =df,x='labOrderResult' )
                         
# fig1.show()

In [ ]:
# selected_lab_df.loc[(selected_lab_df['labItemsNameRef'] == 'WBC'), 'labItemsUnit'].value_counts()

In [ ]:
# selected_lab_df.loc[selected_lab_df['labItemsNameRef'] == 'WBC'].shape[0]

In [ ]:
# selected_lab_df.loc[selected_lab_df['labItemsNameRef'] == 'WBC','labOrderResult'].astype(float)

In [ ]:
# unique_labItemsNameRef.loc[unique_labItemsNameRef['labItemsNameRef']=="WBC"].value_counts()

In [ ]:
# selected_lab_df.loc[(selected_lab_df['labItemsNameRef'] == 'BUN'), 'labItemsUnit'].value_counts()

In [ ]:
# selected_lab_df.loc[(selected_lab_df['labItemsNameRef'] == 'CO2'), 'labItemsUnit'].value_counts()

In [ ]:
# selected_lab_df.loc[(selected_lab_df['labItemsNameRef'] == 'Sodium'), 'labItemsUnit'].value_counts()

In [ ]:
# selected_lab_df.loc[(selected_lab_df['labItemsNameRef'] == 'PLT'), 'labItemsUnit'].value_counts()

In [ ]:
# selected_lab_df.loc[(selected_lab_df['labItemsNameRef'] == 'Chloride'), 'labItemsUnit'].value_counts()

In [ ]:
# selected_lab_df.loc[(selected_lab_df['labItemsNameRef'] == 'MCV'), 'labItemsUnit'].value_counts()

In [ ]:
# selected_lab_df.loc[(selected_lab_df['labItemsNameRef'] == 'MCH'), 'labItemsUnit'].value_counts()

In [ ]:
# selected_lab_df.loc[(selected_lab_df['labItemsNameRef'] == 'MCHC'), 'labItemsUnit'].value_counts()

In [ ]:
# selected_lab_df.loc[(selected_lab_df['labItemsNameRef'] == 'Basophil'), 'labItemsUnit'].value_counts()

In [ ]:
# selected_lab_df.loc[(selected_lab_df['labItemsNameRef'] == 'Neutrophil'), 'labItemsUnit'].value_counts()

In [ ]:
# selected_lab_df.loc[(selected_lab_df['labItemsNameRef'] == 'Eosinophil'), 'labItemsUnit'].value_counts()

In [ ]:
# selected_lab_df.loc[(selected_lab_df['labItemsNameRef'] == 'Monocytes'), 'labItemsUnit'].value_counts()

In [ ]:
# selected_lab_df.loc[(selected_lab_df['labItemsNameRef'] == 'Lymphocytes'), 'labItemsUnit'].value_counts()

In [ ]:
# _selected_lab = ['Creatinine', 'FBS' , 'RBC', 'WBC' ,'LDL-Cholesterol','Potassium','BUN',                 
# 'HCT', 'CO2', 'Sodium' ,'PLT', 'MCV', 'MCH', 'Triglyceride', 'MCHC' ,'Basophil' ,'Neutrophil', 'Eosinophil',
# 'Monocytes' ,'Lymphocytes' ,'Chloride', 'Cholesterol'
#                 ]
# data = df_concat2.loc[df_concat2["labItemsNameRef"].isin(_selected_lab)]

In [ ]:
asas

In [40]:
# change labItemsUnit to the top value_count for the Top 50 lab
for i in df_concat2['labItemsNameRef'].value_counts()[0:50].index:
    df_concat2['labItemsUnit'].loc[df_concat2['labItemsNameRef'] == i] = df_concat2[
        'labItemsUnit'].loc[df_concat2['labItemsNameRef'] == i].value_counts().keys()[0]
    
# change labItemsNormalValueRef to the top value_count for the Top 50 lab
for i in df_concat2['labItemsNameRef'].value_counts()[0:50].index:
    df_concat2['labItemsNormalValueRef'].loc[df_concat2['labItemsNameRef'] == i] = df_concat2[
        'labItemsNormalValueRef'].loc[df_concat2['labItemsNameRef'] == i].value_counts().keys()[0]

In [41]:
# checking
df_concat2[df_concat2['labItemsNameRef'] == 'Creatinine']

,visits,rowID,cid,visitDateTime,labHeadData,departmentName,spcltyName,labItemCode,labItemsNameRef,labOrderResult,labItemsNormalValueRef,labItemsUnit
7,1,100046,273230e0e5855980eff63907d38d2420a03caab239682c35a75247bdd7ef383c,2020-09-24,0,220 หลังพบแพทย์อายุรกรรม,,833,Creatinine,0.78,0.7 - 1.2,mg/dL
21,1,1000750,bac1da024eb599ad388dffd34ac6aa427dbc381da57c8059175b572d0d306ef8,2020-03-02,0,จุดซักประวัติห้องตรวจโรคอายุรกรรม,อายุรกรรม,9,Creatinine,0.57,0.7 - 1.2,mg/dL
56,2,1000757,57626ef6003929e6c1e1c096ddb17c6904287c3b7e235cda4bf9b0bc55f7bf30,2020-03-02,0,จุดซักประวัติห้องตรวจโรคอายุรกรรม,อายุรกรรม,9,Creatinine,0.73,0.7 - 1.2,mg/dL
99,5,1000990,c342837fa4f6369717d051f9a339cae48c97030757a75b742e26674d22470e67,2020-03-02,0,จุดซักประวัติห้องตรวจโรคอายุรกรรม,อายุรกรรม,9,Creatinine,0.99,0.7 - 1.2,mg/dL
110,4,1001224,8d033528c788c66e5286b313ddc9c39c5cebfa873d1689aa9fef0c3ec924a824,2021-08-04,0,ห้องบัตร,อายุรกรรม,78,Creatinine,1.52,0.7 - 1.2,mg/dL
...,...,...,...,...,...,...,...,...,...,...,...,...
650981,4,999443,da6ba27788a42e16cf98bbd16c003986715cd056ef529800c0317d9c5f4b47f7,2020-03-02,0,จุดซักประวัติห้องตรวจโรคอายุรกรรม,อายุรกรรม,9,Creatinine,1.06,0.7 - 1.2,mg/dL
650993,3,999468,941d73f3168654188cc5be163129e395e69801106e49e17be51ba8f0c73750cc,2018-12-20,0,ฝ่ายเวชระเบียน,อายุรกรรม,5073,Creatinine,0.81,0.7 - 1.2,mg/dL
651002,6,999473,941d73f3168654188cc5be163129e395e69801106e49e17be51ba8f0c73750cc,2019-11-21,0,ฝ่ายเวชระเบียน,อายุรกรรมเฉพาะทาง,5073,Creatinine,0.92,0.7 - 1.2,mg/dL
651013,1,999485,941d73f3168654188cc5be163129e395e69801106e49e17be51ba8f0c73750cc,2018-01-18,0,ฝ่ายเวชระเบียน,อายุรกรรมเฉพาะทาง,5073,Creatinine,0.76,0.7 - 1.2,mg/dL


In [ ]:
df_concat2

In [47]:
df_concat2.loc[(df_concat2['labItemsNameRef'] == 'HCT'), 'labOrderResult'].unique()

array(['37.4', '27.6', '38', '26.9', '29.9', '47', '29.4', '35', '42.9',
       '36.9', '37.6', '40', '30', '30.4', '41', '41.1', '41.7', '26',
       '25', '43', '13.5', '14.8', '32.0', '37', '37.2', '40.9', '39',
       '23', '42', '36.3', '38.7', '45.3', '36.0', '42.7', '31', '42.0',
       '32', '29.6', '42.5', '57', '36.7', '46.2', '39.0', '35.8', '39.3',
       '49', '44', '38.8', '41.2', '47.7', '33.5', '32.9', '33.3', '40.0',
       '43.4', '43.5', '33.6', '44.5', '44.2', '36', '44.1', '42.2',
       '31.6', '42.3', '44.7', '41.3', '48', '36.2', '45.7', '28.9',
       '43.6', '45', '44.3', '29.8', '38.2', '40.4', '37.5', '45.0',
       '36.5', '47.1', '44.4', '34.5', '48.3', '46', '28.5', '52.3',
       '26.5', '40.5', '41.0', '35.6', '45.4', '43.8', '48.7', '43.3',
       '44.6', '42.4', '42.8', '40.6', '47.8', '39.1', '41.9', '38.0',
       '40.7', '45.6', '45.1', '49.7', '34', '52.1', '42.1', '36.1',
       '38.3', '52.5', '37.7', '44.9', '34.0', '32.3', '39.8', '33.9',
    

In [46]:
df_concat2.loc[(df_concat2['labItemsNameRef'] == 'HCT') & (df_concat2['labOrderResult'] == 'repeat 48.0'),
                    ['labOrderResult','labItemsNormalValueRef','labItemsUnit']]

,labOrderResult,labItemsNormalValueRef,labItemsUnit


In [45]:
df_concat2['labOrderResult'] = df_concat2['labOrderResult'].apply(lambda x: str(x).replace('repeat' , '').strip())

In [52]:
df_concat2.loc[(df_concat2['labItemsNameRef'] == 'LDL-Cholesterol'), 'labOrderResult'].unique()

array(['85', '67', '71', '51', '204', '48', '40', '82', '174', '165',
       '60', '147', '169', '98', '88', '80', '52', '163', '94', '145',
       '83', '75', '44', '142', '56', '130', '63', '73', '127', '134',
       '64', '117', '97', '122', '92', '177', '101', '104', '103', '95',
       '140', '61', '93', '65', '99', '62', '84', '53', '86', '36', '102',
       '87', '120', '123', '72', '55', '153', '91', '81', '111', '189',
       '57', '79', '58', '106', '78', '77', '107', '90', '70', '100',
       '68', '29', '69', '190', '109', '89', '112', '124', '108', '172',
       '146', '121', '50', '182', '76', '139', '150', '126', '143', '74',
       '115', '154', '42', '37', '207', '194', '66', '128', '193', '110',
       '133', '54', '162', '118', '13', '164', '46', '173', '47', '119',
       '187', '149', '157', '125', '160', '137', '144', '105', '96', '59',
       '43', '49', '132', '39', '113', '34', '223', '188', '141', '114',
       '136', '158', '151', '205', '159', '33', '30', '1

In [51]:
df_concat2['labOrderResult'] = df_concat2['labOrderResult'].apply(lambda x: str(x).replace('LDL-direct=' , '').strip())
df_concat2['labOrderResult'] = df_concat2['labOrderResult'].apply(lambda x: str(x).replace('LDL Direct = ' , '').strip())
df_concat2['labOrderResult'] = df_concat2['labOrderResult'].apply(lambda x: str(x).replace('LDL-Direct = ' , '').strip())
df_concat2['labOrderResult'] = df_concat2['labOrderResult'].apply(lambda x: str(x).replace('LDL-Direct=' , '').strip())

In [56]:
df_concat2.loc[(df_concat2['labItemsNameRef'] == 'FBS'), 'labOrderResult'].unique()

array(['95', '96', '87', '144', '115', '158', '169', '139', '127', '128',
       '131', '103', '152', '155', '108', '179', '117', '136', '126',
       '98', '104', '119', '97', '146', '92', '137', '101', '106',
       'Negative', '110', '233', '262', '251', '145', '154', '60', '91',
       '383', '99', '84', '124', '140', '56', '458', '222', '102', '122',
       '114', '210', '4', '356', '188', '162', '132', '208', '100', '82',
       '79', '86', '94', '112', '134', '109', '125', '190', '116', '113',
       '215', '105', '157', '323', '88', '151', '200', '89', '156', '135',
       '191', '182', '143', '93', '120', '318', '254', '78', '123', '160',
       '148', '167', '142', '121', '90', '227', '228', '138', '80', '111',
       '180', '174', '294', '171', '83', '164', '85', '213', '330', '589',
       '346', '284', '129', '440', '184', '166', '118', '277', '107',
       '133', '141', '150', '130', '159', '198', '185', '149', '178',
       '147', '77', '194', '183', '76', '221', '189', 

In [55]:
df_concat2['labOrderResult'] = df_concat2['labOrderResult'].apply(lambda x: str(x).replace('DTX=' , '').strip())

In [57]:
df_concat2.loc[(df_concat2['labItemsNameRef'] == 'FBS') & (df_concat2['labOrderResult'] == '100'),
                    ['labOrderResult','labItemsNormalValueRef','labItemsUnit']]

,labOrderResult,labItemsNormalValueRef,labItemsUnit
2811,100,74 - 109,mg/dL
8041,100,74 - 109,mg/dL
12405,100,74 - 109,mg/dL
16666,100,74 - 109,mg/dL
17848,100,74 - 109,mg/dL
...,...,...,...
631041,100,74 - 109,mg/dL
639887,100,74 - 109,mg/dL
640488,100,74 - 109,mg/dL
643932,100,74 - 109,mg/dL


In [58]:
df_concat2.loc[(df_concat2['labItemsNameRef'] == 'Creatinine'), 'labOrderResult'].unique()

array(['0.78', '0.57', '0.73', '0.99', '1.52', '1.56', '1.71', '2.80',
       '2.02', '6.56', '0.52', '0.45', '0.93', '0.61', '0.96', '0.60',
       '1.40', '0.92', '0.64', '1.43', '1.47', '1.08', '1.24', '1.17',
       '1', '1.12', '0.1', '1.36', '1.05', '1.19', '1.38', '0.85', '0.86',
       '4.11', '1.81', '10.88', '11.35', '0.83', '1.01', '0.8', '2.18',
       '0.90', '0.9', '1.64', '1.21', '0.81', '1.59', '1.1', '0.84',
       '1.0', '1.54', '0.67', '0.72', '0.97', '1.11', '4.00', '1.42',
       '0.89', '0.95', '0.70', '3.29', '0.76', '1.07', '0.65', '0.77',
       '1.10', '1.31', '1.35', '1.25', '1.50', '1.41', '1.72', '1.51',
       '0.74', '1.04', '0.58', '0.69', '0.56', '0.80', '2.62', '1.82',
       '1.20', '1.32', '1.28', '1.63', '1.85', '1.49', '1.99', '0.91',
       '1.02', '0.75', '1.62', '2.08', '1.13', '1.60', '0.87', '1.55',
       '0.62', '1.06', '0.46', '11.68', '1.00', '0.47', '0.66', '2.91',
       '0.79', '0.82', '1.98', '1.03', '0.53', '0.94', '0.88', '1.09',
   

In [63]:
df_concat2.loc[(df_concat2['labItemsNameRef'] == 'Creatinine') & (df_concat2['labOrderResult'] == '3'),
                    ['labOrderResult','labItemsNormalValueRef','labItemsUnit']]

,labOrderResult,labItemsNormalValueRef,labItemsUnit
29764,3,0.7 - 1.2,mg/dL
80455,3,0.7 - 1.2,mg/dL
112454,3,0.7 - 1.2,mg/dL
120148,3,0.7 - 1.2,mg/dL
284323,3,0.7 - 1.2,mg/dL
332510,3,0.7 - 1.2,mg/dL
340164,3,0.7 - 1.2,mg/dL


In [62]:
df_concat2['labOrderResult'] = df_concat2['labOrderResult'].apply(lambda x: str(x).replace('>' , '').strip())
df_concat2['labOrderResult'] = df_concat2['labOrderResult'].apply(lambda x: str(x).replace('<' , '').strip())

In [66]:
df_concat2.dropna(subset = ['labHeadData'])

,visits,rowID,cid,visitDateTime,labHeadData,departmentName,spcltyName,labItemCode,labItemsNameRef,labOrderResult,labItemsNormalValueRef,labItemsUnit
2,2,1000274,239378b8bc40473f3f73235f6306c0ebb1d50af8aae434de8f75f0bac4fcd742,2020-03-02,0,งานหน้าห้องตรวจโรคทั่วไป 1,อายุรกรรม GP,1001,Glucose,231,74.00 - 106.00,
5,1,100046,273230e0e5855980eff63907d38d2420a03caab239682c35a75247bdd7ef383c,2020-09-24,0,220 หลังพบแพทย์อายุรกรรม,,50,BUN,9,6 - 20,mg/dL
6,1,100046,273230e0e5855980eff63907d38d2420a03caab239682c35a75247bdd7ef383c,2020-09-24,0,220 หลังพบแพทย์อายุรกรรม,,829,Cholesterol,147,< 200,mg/dL
7,1,100046,273230e0e5855980eff63907d38d2420a03caab239682c35a75247bdd7ef383c,2020-09-24,0,220 หลังพบแพทย์อายุรกรรม,,833,Creatinine,0.78,0.7 - 1.2,mg/dL
8,1,100046,273230e0e5855980eff63907d38d2420a03caab239682c35a75247bdd7ef383c,2020-09-24,0,220 หลังพบแพทย์อายุรกรรม,,842,eGFR,103.89,,
...,...,...,...,...,...,...,...,...,...,...,...,...
651094,1,999629,9638a995f2836643b505771c00668f0d6951b1084f10b5735051dd85ffc51534,2020-03-02,0,ฝ่ายเวชระเบียน,ชลอไต,5096,CO2,25.5,21 - 32,mmol/L
651095,1,999629,9638a995f2836643b505771c00668f0d6951b1084f10b5735051dd85ffc51534,2020-03-02,0,ฝ่ายเวชระเบียน,ชลอไต,5286,Elliptocytes,,,
651096,1,999629,9638a995f2836643b505771c00668f0d6951b1084f10b5735051dd85ffc51534,2020-03-02,0,ฝ่ายเวชระเบียน,ชลอไต,5287,Tear drop cell,,,
651097,1,999629,9638a995f2836643b505771c00668f0d6951b1084f10b5735051dd85ffc51534,2020-03-02,0,ฝ่ายเวชระเบียน,ชลอไต,5288,Stomatocytes,,,


In [67]:
top_lab = df_concat2['labItemsNameRef'].value_counts()[:22] # look lab top 22
top_lab

Creatinine         22544
FBS                21228
RBC                16231
WBC                16074
LDL-Cholesterol    14580
Potassium          14578
BUN                13470
HCT                13172
CO2                13147
Sodium             12513
PLT                12279
MCV                11948
MCH                11947
Triglyceride       11752
MCHC               11497
Basophil           10938
Neutrophil         10928
Lymphocytes        10920
Monocytes          10920
Eosinophil         10920
Chloride           10315
Cholesterol        9883 
Name: labItemsNameRef, dtype: int64

In [68]:
# _selected_lab = ['Creatinine', 'FBS' , 'RBC', 'WBC' ,'LDL-Cholesterol','Potassium','BUN',                 
# 'HCT', 'CO2', 'Sodium' ,'PLT', 'MCV', 'MCH', 'Triglyceride', 'MCHC' ,'Basophil' ,'Neutrophil', 'Eosinophil',
# 'Monocytes' ,'Lymphocytes' ,'Chloride', 'Cholesterol' ,'eGFR' ,'PT', 'HDL-Cholesterol', 'eGFR (CKD-EPI)',
# 'eGFR (MDRD)', 'INR', 'HbA1c', 'HGB', 'SGPT', 'SGOT', 'Absolute', 'Neutrophil', 'Albumin', 'DTX'
#                 ]
# data = df_concat2.loc[df_concat2["labItemsNameRef"].isin(_selected_lab)]

In [69]:
_selected_lab = ['Creatinine', 'FBS' , 'RBC', 'WBC' ,'LDL-Cholesterol','Potassium','BUN',                 
'HCT', 'CO2', 'Sodium' ,'PLT', 'MCV', 'MCH', 'Triglyceride', 'MCHC' ,'Basophil' ,'Neutrophil', 'Eosinophil',
'Monocytes' ,'Lymphocytes' ,'Chloride', 'Cholesterol'
                ]
data = df_concat2.loc[df_concat2["labItemsNameRef"].isin(_selected_lab)]

In [70]:
data

,visits,rowID,cid,visitDateTime,labHeadData,departmentName,spcltyName,labItemCode,labItemsNameRef,labOrderResult,labItemsNormalValueRef,labItemsUnit
5,1,100046,273230e0e5855980eff63907d38d2420a03caab239682c35a75247bdd7ef383c,2020-09-24,0,220 หลังพบแพทย์อายุรกรรม,,50,BUN,9,6 - 20,mg/dL
6,1,100046,273230e0e5855980eff63907d38d2420a03caab239682c35a75247bdd7ef383c,2020-09-24,0,220 หลังพบแพทย์อายุรกรรม,,829,Cholesterol,147,< 200,mg/dL
7,1,100046,273230e0e5855980eff63907d38d2420a03caab239682c35a75247bdd7ef383c,2020-09-24,0,220 หลังพบแพทย์อายุรกรรม,,833,Creatinine,0.78,0.7 - 1.2,mg/dL
9,1,100046,273230e0e5855980eff63907d38d2420a03caab239682c35a75247bdd7ef383c,2020-09-24,0,220 หลังพบแพทย์อายุรกรรม,,846,FBS,95,74 - 109,mg/dL
11,1,100046,273230e0e5855980eff63907d38d2420a03caab239682c35a75247bdd7ef383c,2020-09-24,0,220 หลังพบแพทย์อายุรกรรม,,860,LDL-Cholesterol,85,< 100,mg/dL
...,...,...,...,...,...,...,...,...,...,...,...,...
651090,1,999629,9638a995f2836643b505771c00668f0d6951b1084f10b5735051dd85ffc51534,2020-03-02,0,ฝ่ายเวชระเบียน,ชลอไต,5073,Creatinine,1.53,0.7 - 1.2,mg/dL
651091,1,999629,9638a995f2836643b505771c00668f0d6951b1084f10b5735051dd85ffc51534,2020-03-02,0,ฝ่ายเวชระเบียน,ชลอไต,5093,Sodium,140.3,136 - 145,mmol/L
651092,1,999629,9638a995f2836643b505771c00668f0d6951b1084f10b5735051dd85ffc51534,2020-03-02,0,ฝ่ายเวชระเบียน,ชลอไต,5094,Potassium,4.05,3.5 - 5.1,mmol/L
651093,1,999629,9638a995f2836643b505771c00668f0d6951b1084f10b5735051dd85ffc51534,2020-03-02,0,ฝ่ายเวชระเบียน,ชลอไต,5095,Chloride,102.8,98 - 107,mmol/L


In [71]:
data.loc[(data['labItemsNameRef'] == 'Triglyceride')  ,
         ['labOrderResult','labItemsNormalValueRef','labItemsUnit']].sort_values('labOrderResult')[:50]

,labOrderResult,labItemsNormalValueRef,labItemsUnit
208229,,< 200,mg/dL
624007,,< 200,mg/dL
282701,,< 200,mg/dL
407791,,< 200,mg/dL
294560,,< 200,mg/dL
149969,,< 200,mg/dL
364958,,< 200,mg/dL
160806,,< 200,mg/dL
171528,-,< 200,mg/dL
49237,"1,114",< 200,mg/dL


In [72]:
labData = data.loc[:, ["rowID", "cid", "labItemsNameRef", "labOrderResult"]]
labData

,rowID,cid,labItemsNameRef,labOrderResult
5,100046,273230e0e5855980eff63907d38d2420a03caab239682c35a75247bdd7ef383c,BUN,9
6,100046,273230e0e5855980eff63907d38d2420a03caab239682c35a75247bdd7ef383c,Cholesterol,147
7,100046,273230e0e5855980eff63907d38d2420a03caab239682c35a75247bdd7ef383c,Creatinine,0.78
9,100046,273230e0e5855980eff63907d38d2420a03caab239682c35a75247bdd7ef383c,FBS,95
11,100046,273230e0e5855980eff63907d38d2420a03caab239682c35a75247bdd7ef383c,LDL-Cholesterol,85
...,...,...,...,...
651090,999629,9638a995f2836643b505771c00668f0d6951b1084f10b5735051dd85ffc51534,Creatinine,1.53
651091,999629,9638a995f2836643b505771c00668f0d6951b1084f10b5735051dd85ffc51534,Sodium,140.3
651092,999629,9638a995f2836643b505771c00668f0d6951b1084f10b5735051dd85ffc51534,Potassium,4.05
651093,999629,9638a995f2836643b505771c00668f0d6951b1084f10b5735051dd85ffc51534,Chloride,102.8


In [73]:
# labData.loc[(labData["labItemsNameRef"] == "Urine-sugar")
#         &(labData["labOrderResult"].isin([""])), 
#         "labOrderResult"] = np.NaN

# labData.loc[(labData["labItemsNameRef"] == "Urine-sugar")
#         & (labData["labOrderResult"].str.contains("n|N|ปกติ|-|Trace|trace")) 
#         & (labData["labItemsNameRef"].notnull()), 
#         "labOrderResult"] = 0

# labData.loc[(labData["labItemsNameRef"] == "Urine-sugar")
#         & (labData["labOrderResult"].str.contains("1")) 
#         & (labData["labItemsNameRef"].notnull()), 
#         "labOrderResult"] = 1

# labData.loc[(labData["labItemsNameRef"] == "Urine-sugar")
#         & (labData["labOrderResult"].str.contains("2")) 
#         & (labData["labItemsNameRef"].notnull()), 
#         "labOrderResult"] = 2

# labData.loc[(labData["labItemsNameRef"] == "Urine-sugar")
#         & (labData["labOrderResult"].str.contains("3")) 
#         & (labData["labItemsNameRef"].notnull()), 
#         "labOrderResult"] = 3

# labData.loc[(labData["labItemsNameRef"] == "Urine-sugar")
#         & (labData["labOrderResult"].str.contains("4")) 
#         & (labData["labItemsNameRef"].notnull()), 
#         "labOrderResult"] = 4


# labData.loc[(labData["labItemsNameRef"] == "Urine-protein-pn")
#         &(labData["labOrderResult"].isin([""])), 
#         "labOrderResult"] = np.NaN

# labData.loc[(labData["labItemsNameRef"] == "Urine-protein-pn")
#         & (labData["labOrderResult"].str.contains("n|N|ปกติ|-|Trace|trace")) 
#         & (labData["labItemsNameRef"].notnull()), 
#         "labOrderResult"] = 0

# labData.loc[(labData["labItemsNameRef"] == "Urine-protein-pn")
#         & (labData["labOrderResult"].str.contains("1")) 
#         & (labData["labItemsNameRef"].notnull()), 
#         "labOrderResult"] = 1

# labData.loc[(labData["labItemsNameRef"] == "Urine-protein-pn")
#         & (labData["labOrderResult"].str.contains("2")) 
#         & (labData["labItemsNameRef"].notnull()), 
#         "labOrderResult"] = 2

# labData.loc[(labData["labItemsNameRef"] == "Urine-protein-pn")
#         & (labData["labOrderResult"].str.contains("3")) 
#         & (labData["labItemsNameRef"].notnull()), 
#         "labOrderResult"] = 3

# labData.loc[(labData["labItemsNameRef"] == "Urine-protein-pn")
#         & (labData["labOrderResult"].str.contains("4")) 
#         & (labData["labItemsNameRef"].notnull()), 
#         "labOrderResult"] = 4

# labData.loc[(labData["labItemsNameRef"] == "Pregnancy Test")
#         &(labData["labOrderResult"].isin(["", "***Pending result***"])), 
#         "labOrderResult"] = np.NaN

# labData.loc[(labData["labItemsNameRef"] == "Pregnancy Test")
#         & (labData["labOrderResult"].str.contains("n|N|ปกติ|-")) 
#         & (labData["labItemsNameRef"].notnull()), 
#         "labOrderResult"] = 0

# # แถวไหนมี Pregnancy Test และ มี P หรือ p นำหน้า และไม่เป็นค่าว่าง จะ assign ให้ result เป็น 1
# labData.loc[(labData["labItemsNameRef"] == "Pregnancy Test")
#         & (labData["labOrderResult"].str.contains("P|p")) 
#         & (labData["labItemsNameRef"].notnull()), 
#         "labOrderResult"] = 1

labData["labOrderResult"] = labData["labOrderResult"].apply(
    lambda x: pd.to_numeric(x, errors= "coerce"))

labData = pl.from_pandas(labData)
# columns= "labName"   values= "labResult" index= "rowID"
labData = labData.pivot(values= "labOrderResult", columns= "labItemsNameRef", index= "rowID", aggregate_fn= "max")
labData = labData.to_pandas()

In [74]:
labData

,rowID,BUN,Cholesterol,Creatinine,FBS,LDL-Cholesterol,Triglyceride,Sodium,Potassium,Chloride,CO2,HCT,WBC,RBC,PLT,Neutrophil,Lymphocytes,Monocytes,Eosinophil,Basophil,MCV,MCH,MCHC
0,100046,9.00,147.00,0.78,95.00,85.00,50.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000750,NaN,NaN,0.57,96.00,NaN,NaN,141.00,4.12,105.00,23.60,37.40,7.60,4.20,302.00,72.20,18.50,6.60,1.90,0.80,89.00,29.00,32.60
2,1000757,7.90,NaN,0.73,87.00,67.00,80.00,136.00,2.89,97.00,24.00,27.60,8.40,3.75,673.00,73.50,20.20,4.40,1.30,0.60,73.60,23.20,31.50
3,1000990,NaN,NaN,0.99,144.00,71.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1001224,19.90,116.00,1.52,NaN,51.00,205.00,NaN,NaN,NaN,25.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30820,999443,NaN,249.00,1.06,410.00,184.00,114.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30821,999468,NaN,193.00,0.81,81.00,92.00,95.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30822,999473,NaN,173.00,0.92,82.00,74.00,151.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30823,999485,NaN,172.00,0.76,101.00,115.00,108.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
data = df_concat2.iloc[:, 0:-2].drop_duplicates(subset= "rowID") # หลัง pivot

In [76]:
data = pd.merge(data_col, labData, how= "left", left_on= "rowID", right_on="rowID")

In [77]:
data

,visits,rowID,cid,vital_cc,vital_pe,visitDateTime,icd10_list,diagType_1,diagType_2,diagType_3,patientAge,patientSexName,smoking,narcotic,exercise,alcohol,vital_bw,vital_waist,vital_height,vital_bmi,vital_hr,vital_pulse,vital_temperature,vital_rr,vital_bps,vital_bpd,BUN,Cholesterol,Creatinine,FBS,LDL-Cholesterol,Triglyceride,Sodium,Potassium,Chloride,CO2,HCT,WBC,RBC,PLT,Neutrophil,Lymphocytes,Monocytes,Eosinophil,Basophil,MCV,MCH,MCHC
0,2,1000135,b472531d4caaecc893dc345f31ee8317ea459926dd37f0da1c50d2af2e8543aa,case Acute ischemic stroke (Lt. hemiparesthesia)\rนัดติดตามอาการ ไม่มีอาการผิดปกติ,08.30 น. EKG 12 leed // พว. หนึ่งฤทัย อุดารักษ์,2022-08-16,[I639],[I639],[],[],52,,NA,NA,NA,NA,71,0,160,27.73,74,74,36.6,20,136,84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1000246,1cf272b413de5fc4e1166e3b5aa35e146125efd0f33ad13a73f58368da638167,"Case Intracerebral haemorrhage in hemisphere, subcortical / Hyperlipidaemia, unspecified / Essential (primary) hypertension / มาตามนัด, อาการทั่วไปปกติ, ทานยาสม่ำเสมอ, ปฏิเสธการดื่มสุรา/การสูบบุหรี่/การใช้สารเสพติด,",None,2020-03-02,"[I610, I10, E785]",[I610],"[I10, E785]",[],53,1.0,No,No,None,No,82.0,0.0,0.0,400.0,None,76.0,36.0,20.0,112.0,66.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1000274,239378b8bc40473f3f73235f6306c0ebb1d50af8aae434de8f75f0bac4fcd742,"06.45 น. ผู้ป่วยให้ประวัติว่า.... ติดตามอาการ + FBSก่อนพบแพทย์..... พูดชัดเจนดี ไม่มีแขนขาอ่อนแรง คืนนี้เจ็บหน้าอก อมยาใต้ลิ้นทุเลา ไม่มีหอบเหยนื่อย \r\nMEWS=0\r\n\r\nโรคประจำตัว DM,DLP,HT,เส้นเลือดหัวใจตีบ 2 เส้น /แพ้ยา Penicillin/ผ่าตัด PCI 2 เส้น\r\n/แนะนำหากอาการไม่ดีขึ้นให้กลับมาตรวจซ้ำ","case ชาย 73 ปี DM, HT, DLP, DVD S/P PCI\r\nadmit 14-16/2/63 Dx R/O TIA \r\nCT brain : - A small ill-defined hypodense lesion at anterior limb of right internal capsule and right caudate head, probably acute lacunar infarction.\r\n\r\nPE : V/S stable \r\nDTX = 149\r\nPE : motor grade V all, sensory intact \r\nFBS 231 --> นำผล ไปปรับยาที่รพ.ท่าเรือ \r\nRM + dec ASA \r\n\r\nadvice รักษารพ.ใดรพ.หนึ่ง เพื่อให้ง่ายต่อการปรับยา",2020-03-02,"[I64, E149, I10, I251, E789]",[I64],"[E149, I10, I251, E789]",[],73,1.0,None,None,None,None,68.0,0.0,156.0,27.942,None,95.0,37.0,20.0,124.0,72.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,1000311,b360f3bf1549170ef3d569c7f618f004bd2b78d210b8c7f80000ba75718cf01c,case HT Stroke DLP loss F/U 31/5/65 \rให้ประวัติว่าทานยาทุกวัน ปวดศีรษะบ่อย,,2022-08-01,"[I64, I10, R42]",[I64],"[I10, R42]",[],51,,NA,NA,NA,NA,72.7,87,165,26.7,74,74,37,20,147,80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,100039,4bad23f241ec47aeb6f333bbc87a7b4da1b6ded6969d21789ddb638be41ea985,ญาติ รับาเดิม อ. สุวิช แทน,None,2021-01-14,[I639],[I639],[],[],75,1.0,None,None,None,None,None,None,163.0,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111006,9,999616,9ad207041caafe6845f93e05f4e4fffc1d6cd521ad462a1ec624f0e2b029828c,ผู้ป่วยอ่อนแรงซ้าย ปวดไหล่ขวา เป็นมา 2 ปี,None,2018-12-12,"[I694, U78112]",[I694],[],[],51,0.0,None,None,None,None,50.0,None,154.0,None,None,70.0,37.0,20.0,135.0,65.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111007,19,999618,9ad207041caafe6845f93e05f4e4fffc1d6cd521ad462a1ec624f0e2b029828c,", บุตรให้ประวัติ, ไปรับยาตามนัดที่คลินิกตรวจพบ BP ต่ำแนะนมารพ. ไม่มีอาการผิดปกติ",alert\r\nno pale \r\npulse full\r\nheart and lung normla\r\ncap refil < 2 sec\r\nBP at ER 97/59 MAP 72 clinical stable \r\nกลับบ้าน \r\nวันนี้มาเช็คยาที่รับจาก พุทธโสธร เนื่องจากต้องการมารับต่อที่ รพช ไม่มี depakin(500),2020-04-14,[I693],[I693],[],[],52,0.0,None,None,None,None,

In [78]:
# เอาเฉพาะ diagType_1 contanis I6 
data = data.explode('diagType_1').reset_index(drop=True)
data = data[(data['diagType_1'] .str.contains("I6"))]
data

,visits,rowID,cid,vital_cc,vital_pe,visitDateTime,icd10_list,diagType_1,diagType_2,diagType_3,patientAge,patientSexName,smoking,narcotic,exercise,alcohol,vital_bw,vital_waist,vital_height,vital_bmi,vital_hr,vital_pulse,vital_temperature,vital_rr,vital_bps,vital_bpd,BUN,Cholesterol,Creatinine,FBS,LDL-Cholesterol,Triglyceride,Sodium,Potassium,Chloride,CO2,HCT,WBC,RBC,PLT,Neutrophil,Lymphocytes,Monocytes,Eosinophil,Basophil,MCV,MCH,MCHC
0,2,1000135,b472531d4caaecc893dc345f31ee8317ea459926dd37f0da1c50d2af2e8543aa,case Acute ischemic stroke (Lt. hemiparesthesia)\rนัดติดตามอาการ ไม่มีอาการผิดปกติ,08.30 น. EKG 12 leed // พว. หนึ่งฤทัย อุดารักษ์,2022-08-16,[I639],I639,[],[],52,,NA,NA,NA,NA,71,0,160,27.73,74,74,36.6,20,136,84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1000246,1cf272b413de5fc4e1166e3b5aa35e146125efd0f33ad13a73f58368da638167,"Case Intracerebral haemorrhage in hemisphere, subcortical / Hyperlipidaemia, unspecified / Essential (primary) hypertension / มาตามนัด, อาการทั่วไปปกติ, ทานยาสม่ำเสมอ, ปฏิเสธการดื่มสุรา/การสูบบุหรี่/การใช้สารเสพติด,",None,2020-03-02,"[I610, I10, E785]",I610,"[I10, E785]",[],53,1.0,No,No,None,No,82.0,0.0,0.0,400.0,None,76.0,36.0,20.0,112.0,66.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1000274,239378b8bc40473f3f73235f6306c0ebb1d50af8aae434de8f75f0bac4fcd742,"06.45 น. ผู้ป่วยให้ประวัติว่า.... ติดตามอาการ + FBSก่อนพบแพทย์..... พูดชัดเจนดี ไม่มีแขนขาอ่อนแรง คืนนี้เจ็บหน้าอก อมยาใต้ลิ้นทุเลา ไม่มีหอบเหยนื่อย \r\nMEWS=0\r\n\r\nโรคประจำตัว DM,DLP,HT,เส้นเลือดหัวใจตีบ 2 เส้น /แพ้ยา Penicillin/ผ่าตัด PCI 2 เส้น\r\n/แนะนำหากอาการไม่ดีขึ้นให้กลับมาตรวจซ้ำ","case ชาย 73 ปี DM, HT, DLP, DVD S/P PCI\r\nadmit 14-16/2/63 Dx R/O TIA \r\nCT brain : - A small ill-defined hypodense lesion at anterior limb of right internal capsule and right caudate head, probably acute lacunar infarction.\r\n\r\nPE : V/S stable \r\nDTX = 149\r\nPE : motor grade V all, sensory intact \r\nFBS 231 --> นำผล ไปปรับยาที่รพ.ท่าเรือ \r\nRM + dec ASA \r\n\r\nadvice รักษารพ.ใดรพ.หนึ่ง เพื่อให้ง่ายต่อการปรับยา",2020-03-02,"[I64, E149, I10, I251, E789]",I64,"[E149, I10, I251, E789]",[],73,1.0,None,None,None,None,68.0,0.0,156.0,27.942,None,95.0,37.0,20.0,124.0,72.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,1000311,b360f3bf1549170ef3d569c7f618f004bd2b78d210b8c7f80000ba75718cf01c,case HT Stroke DLP loss F/U 31/5/65 \rให้ประวัติว่าทานยาทุกวัน ปวดศีรษะบ่อย,,2022-08-01,"[I64, I10, R42]",I64,"[I10, R42]",[],51,,NA,NA,NA,NA,72.7,87,165,26.7,74,74,37,20,147,80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,100039,4bad23f241ec47aeb6f333bbc87a7b4da1b6ded6969d21789ddb638be41ea985,ญาติ รับาเดิม อ. สุวิช แทน,None,2021-01-14,[I639],I639,[],[],75,1.0,None,None,None,None,None,None,163.0,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111022,9,999616,9ad207041caafe6845f93e05f4e4fffc1d6cd521ad462a1ec624f0e2b029828c,ผู้ป่วยอ่อนแรงซ้าย ปวดไหล่ขวา เป็นมา 2 ปี,None,2018-12-12,"[I694, U78112]",I694,[],[],51,0.0,None,None,None,None,50.0,None,154.0,None,None,70.0,37.0,20.0,135.0,65.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111023,19,999618,9ad207041caafe6845f93e05f4e4fffc1d6cd521ad462a1ec624f0e2b029828c,", บุตรให้ประวัติ, ไปรับยาตามนัดที่คลินิกตรวจพบ BP ต่ำแนะนมารพ. ไม่มีอาการผิดปกติ",alert\r\nno pale \r\npulse full\r\nheart and lung normla\r\ncap refil < 2 sec\r\nBP at ER 97/59 MAP 72 clinical stable \r\nกลับบ้าน \r\nวันนี้มาเช็คยาที่รับจาก พุทธโสธร เนื่องจากต้องการมารับต่อที่ รพช ไม่มี depakin(500),2020-04-14,[I693],I693,[],[],52,0.0,None,None,None,None,50.0,0.0,154.0

In [79]:
data = data.sort_values(by='visitDateTime', ascending=True)

In [80]:
data

,visits,rowID,cid,vital_cc,vital_pe,visitDateTime,icd10_list,diagType_1,diagType_2,diagType_3,patientAge,patientSexName,smoking,narcotic,exercise,alcohol,vital_bw,vital_waist,vital_height,vital_bmi,vital_hr,vital_pulse,vital_temperature,vital_rr,vital_bps,vital_bpd,BUN,Cholesterol,Creatinine,FBS,LDL-Cholesterol,Triglyceride,Sodium,Potassium,Chloride,CO2,HCT,WBC,RBC,PLT,Neutrophil,Lymphocytes,Monocytes,Eosinophil,Basophil,MCV,MCH,MCHC
104769,1,7629,4a1d93a977f66de044778ad54463bc516908785fd67ebe6e6299803733bdc623,None,None,2013-08-02,"[I639, E119, I10, E789]",I639,"[E119, I10, E789]",[],57,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109707,1,94691,ee9c51648123950c3110a83355c6d2fdeb15b4a7b0995bad5f438fc9147a918f,None,None,2014-02-13,"[I635, E876, E789, E162, J029]",I635,"[E876, E789, E162, J029]",[],83,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14530,1,18029,6b22580b026e54c109844361fddb1492cc0c16aa96cce6b09a45b1776eed5399,None,None,2014-03-24,"[I633, G819, E119, I10, E785, M1009]",I633,"[G819, E119, I10, E785, M1009]",[],60,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10430,1,1572858,5379c7d14c13bd043ef5476ff887310345280b79578c0be5d618fcbe29ae77ad,None,None,2015-01-09,[I694],I694,[],[],63,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34430,1,295810,eedebeecd8ed3c0dcf87872fa622268cff199fcfbf40c5707b691278f2df9ded,None,None,2015-01-19,"[I694, E119, I10]",I694,"[E119, I10]",[],81,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103502,3,7522927,8b790826af2f57d43f2011a05cfb2975a13683f47bfcbf572fadff0e8a2516d5,None,"Hx of DM, HT, Old CVA about 10 yr ago, add NPH 10 u OD for 3 yr. \r\n- ติด APEC เลื่อนนัด",2022-11-17,"[I693, E117, I10, H251, H544]",I693,"[E117, I10, H251, H544]",[],64,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103503,9,7522949,c26697771889db270b4708dbe98f68667773e98f09e11409b8619a6da70162a4,None,"1. T2DM\r\nFBS 163 HbA1c 7.3 (initial 13-14) --> 9.4 (21/4/65) \r\nทานข้าวเยอะ \r\ncomplications: ischemic stroke 2555 now Lt weak gr.0, bedbound, DR loss f/u\r\nincrease mixtard\r\n2. DLP LDL 102--> 114 , TG 189\r\n simvas to 40 mg/d\r\n3. CKD stage 2 stable,l UPCr >300",2022-11-17,"[I698, E119, E785, I10]",I698,"[E119, E785, I10]",[],65,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104224,15,7563561,b6cd79f852b3c34d8ff363efcf9351a7d4f73890c30237cc144c873b94dffef3,None,None,2022-11-17,"[I64, E876]",I64,[E876],[],56,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103342,13,7515954,d8f7c093acbdf5e05494de602301fe77e22e87cdbb30469a7cbbeb69b0aa5228,None,1.DM\r\nFBS 132 \r\n2. DLP\r\n3. HT \r\nBP 128/50\r\nannual lab 2/12/64 \r\nนัด 3 month +FBS+HbA1c\r\n\r\n18/11/65 รับยาไม่พบแพทย์ (วันหยุดพิเศษ),2022-11-18,"[I698, E116, E785, I10, M170]",I698,"[E116, E785, I10, M170]",[],65,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [81]:
# คนไข้มากี่ครั้ง
data['cid'].value_counts().reset_index(name='counts')

,index,counts
0,b2f2bdc360326c5753950eda4644c404b71b6ad3ae1e186550dd4b610c0a0122,161
1,7aae1d8f72fff753f4e2a5c82a25277c02ff0f6116ecee1d7bd24a4b60c5edca,115
2,0ad007fce7c50ab27f3c5c030c7c054492d6ebd6096f7be2070b4f25c96a0453,112
3,3b72ef1dc67d7b91ea8a1f46db08b2ba213ceb00e70ec07a6a7882a040596bd5,109
4,09181e7e952c3efb945fe3105347dc24ca3c1413040a848b00dd91b2fab51b31,107
...,...,...
23565,c07b72dda3e6462c78719fe2304aca28478ba0266349c6fe925c22876ce38cf6,1
23566,c6e151afce1c5679f480e5f2f622c3b759b81c6ba094f4cf6b73dc995549f654,1
23567,854329d56611d92e6735110b502af1e1f577fbac73af64bd957528882cfccb4e,1
23568,5f16e72c420a80aebe6762a068e1e369437a0dc5d8e6c59cdef5a7ff5fc949b3,1


In [82]:
data['diagType_1'].value_counts()

I64      38055
I693     18284
I639     17302
I698     11195
I694     4960 
I633     4564 
I635     2451 
I610     2410 
I619     2282 
I638     1649 
I691     1370 
I634     743  
I692     726  
I679     501  
I620     500  
I629     475  
I678     349  
I630     345  
I611     312  
I613     256  
I618     249  
I636     244  
I612     229  
I614     179  
I632     176  
I671     161  
I676     150  
I615     145  
I631     106  
I690     90   
I674     81   
I616     74   
I675     65   
I69      45   
I652     40   
I672     37   
I688     31   
I669     26   
I660     25   
I662     20   
I63      16   
I61      14   
I621     11   
I670     10   
I677     8    
I680     8    
I663     8    
I659     6    
I668     5    
I62      5    
I653     5    
I681     4    
I67      2    
I68      2    
I650     2    
I682     2    
I661     2    
I651     1    
I63.1    1    
I63.9    1    
Name: diagType_1, dtype: int64

In [83]:
data

,visits,rowID,cid,vital_cc,vital_pe,visitDateTime,icd10_list,diagType_1,diagType_2,diagType_3,patientAge,patientSexName,smoking,narcotic,exercise,alcohol,vital_bw,vital_waist,vital_height,vital_bmi,vital_hr,vital_pulse,vital_temperature,vital_rr,vital_bps,vital_bpd,BUN,Cholesterol,Creatinine,FBS,LDL-Cholesterol,Triglyceride,Sodium,Potassium,Chloride,CO2,HCT,WBC,RBC,PLT,Neutrophil,Lymphocytes,Monocytes,Eosinophil,Basophil,MCV,MCH,MCHC
104769,1,7629,4a1d93a977f66de044778ad54463bc516908785fd67ebe6e6299803733bdc623,None,None,2013-08-02,"[I639, E119, I10, E789]",I639,"[E119, I10, E789]",[],57,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109707,1,94691,ee9c51648123950c3110a83355c6d2fdeb15b4a7b0995bad5f438fc9147a918f,None,None,2014-02-13,"[I635, E876, E789, E162, J029]",I635,"[E876, E789, E162, J029]",[],83,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14530,1,18029,6b22580b026e54c109844361fddb1492cc0c16aa96cce6b09a45b1776eed5399,None,None,2014-03-24,"[I633, G819, E119, I10, E785, M1009]",I633,"[G819, E119, I10, E785, M1009]",[],60,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10430,1,1572858,5379c7d14c13bd043ef5476ff887310345280b79578c0be5d618fcbe29ae77ad,None,None,2015-01-09,[I694],I694,[],[],63,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34430,1,295810,eedebeecd8ed3c0dcf87872fa622268cff199fcfbf40c5707b691278f2df9ded,None,None,2015-01-19,"[I694, E119, I10]",I694,"[E119, I10]",[],81,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103502,3,7522927,8b790826af2f57d43f2011a05cfb2975a13683f47bfcbf572fadff0e8a2516d5,None,"Hx of DM, HT, Old CVA about 10 yr ago, add NPH 10 u OD for 3 yr. \r\n- ติด APEC เลื่อนนัด",2022-11-17,"[I693, E117, I10, H251, H544]",I693,"[E117, I10, H251, H544]",[],64,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103503,9,7522949,c26697771889db270b4708dbe98f68667773e98f09e11409b8619a6da70162a4,None,"1. T2DM\r\nFBS 163 HbA1c 7.3 (initial 13-14) --> 9.4 (21/4/65) \r\nทานข้าวเยอะ \r\ncomplications: ischemic stroke 2555 now Lt weak gr.0, bedbound, DR loss f/u\r\nincrease mixtard\r\n2. DLP LDL 102--> 114 , TG 189\r\n simvas to 40 mg/d\r\n3. CKD stage 2 stable,l UPCr >300",2022-11-17,"[I698, E119, E785, I10]",I698,"[E119, E785, I10]",[],65,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104224,15,7563561,b6cd79f852b3c34d8ff363efcf9351a7d4f73890c30237cc144c873b94dffef3,None,None,2022-11-17,"[I64, E876]",I64,[E876],[],56,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103342,13,7515954,d8f7c093acbdf5e05494de602301fe77e22e87cdbb30469a7cbbeb69b0aa5228,None,1.DM\r\nFBS 132 \r\n2. DLP\r\n3. HT \r\nBP 128/50\r\nannual lab 2/12/64 \r\nนัด 3 month +FBS+HbA1c\r\n\r\n18/11/65 รับยาไม่พบแพทย์ (วันหยุดพิเศษ),2022-11-18,"[I698, E116, E785, I10, M170]",I698,"[E116, E785, I10, M170]",[],65,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [84]:
# a = (data[(data['cid'] == '8d033528c788c66e5286b313ddc9c39c5cebfa873d1689aa9fef0c3ec924a824')][['visitDateTime']].max()[0] - 
#      data[(data['cid'] == '8d033528c788c66e5286b313ddc9c39c5cebfa873d1689aa9fef0c3ec924a824')][['visitDateTime']].min()[0]).days
# a

In [85]:
# %%time
# # ไล่ตามเเต่ละ row เเล้วคำนวน max-min ของ DateTime ออกมา เพื่อหาจำนวนปี
# data['year'] = [0] * data_c.shape[0]
# for i,r in data.iterrows():  # i:idex ,r:row 
#     data.at[i,'year'] = int((data[(data['cid'] == r['cid'])][['visitDateTime']].max()[0] -
#                              data[(data['cid'] == r['cid'])][['visitDateTime']].min()[0]).days/365) 

In [86]:
%%time
# สร้าง col ใหม่ขึ้นมาเพื่อเก็บค่า max , min ของ visitDateTime
# agg รวม
df_agg = data.loc[:,['cid','visitDateTime']].groupby(["cid"]).agg(['min','max']).droplevel(0,axis='columns')
df_agg

Wall time: 88.3 ms


,min,max
cid,,
000564176fadea8061ea995f569a5fe56920585ddf3d36cbe5d87b9d0814754f,2020-01-20,2021-04-19
0006c55f2df5302b02c9ec83be0482a934e5f06499acbe9518ebd53565eafb53,2017-03-23,2017-07-20
0007348aefd52845d87e2cada5f2c2655760f303fb714f5f4c776325fa64af6f,2019-11-02,2019-12-26
000d7905b7330e9e6987b50f3934088fb15d33dfe28e45f4c56e4b324419b66d,2017-01-16,2017-01-16
0010a47c034d3e75b90f49df1d04bcba692b7db4ec7465fda95e9454a7897309,2019-01-14,2019-08-16
...,...,...
fff2fb8e80d6feb8a1f6df689ebf0cc9479ebdc6e9bc493f3da88cfa0f627b2f,2021-10-31,2022-10-03
fff37e13ba46cf83d9d0e0a72ac48d94731dbd67a1cc4164c2d16a8408fc4934,2022-01-11,2022-09-08
fff3a3a6ea436b2dbcd62e36fdc4c4d67a89bc22dde43fcbff5da60c945cc757,2022-05-26,2022-05-26


In [87]:
# merge กับ data frame
data = pd.merge(data , df_agg, how = 'left' , on=['cid'])

In [88]:
data

,visits,rowID,cid,vital_cc,vital_pe,visitDateTime,icd10_list,diagType_1,diagType_2,diagType_3,patientAge,patientSexName,smoking,narcotic,exercise,alcohol,vital_bw,vital_waist,vital_height,vital_bmi,vital_hr,vital_pulse,vital_temperature,vital_rr,vital_bps,vital_bpd,BUN,Cholesterol,Creatinine,FBS,LDL-Cholesterol,Triglyceride,Sodium,Potassium,Chloride,CO2,HCT,WBC,RBC,PLT,Neutrophil,Lymphocytes,Monocytes,Eosinophil,Basophil,MCV,MCH,MCHC,min,max
0,1,7629,4a1d93a977f66de044778ad54463bc516908785fd67ebe6e6299803733bdc623,None,None,2013-08-02,"[I639, E119, I10, E789]",I639,"[E119, I10, E789]",[],57,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-08-02,2021-10-04
1,1,94691,ee9c51648123950c3110a83355c6d2fdeb15b4a7b0995bad5f438fc9147a918f,None,None,2014-02-13,"[I635, E876, E789, E162, J029]",I635,"[E876, E789, E162, J029]",[],83,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-02-13,2022-10-31
2,1,18029,6b22580b026e54c109844361fddb1492cc0c16aa96cce6b09a45b1776eed5399,None,None,2014-03-24,"[I633, G819, E119, I10, E785, M1009]",I633,"[G819, E119, I10, E785, M1009]",[],60,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-03-24,2022-10-04
3,1,1572858,5379c7d14c13bd043ef5476ff887310345280b79578c0be5d618fcbe29ae77ad,None,None,2015-01-09,[I694],I694,[],[],63,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-09,2015-01-09
4,1,295810,eedebeecd8ed3c0dcf87872fa622268cff199fcfbf40c5707b691278f2df9ded,None,None,2015-01-19,"[I694, E119, I10]",I694,"[E119, I10]",[],81,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-19,2018-06-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111010,3,7522927,8b790826af2f57d43f2011a05cfb2975a13683f47bfcbf572fadff0e8a2516d5,None,"Hx of DM, HT, Old CVA about 10 yr ago, add NPH 10 u OD for 3 yr. \r\n- ติด APEC เลื่อนนัด",2022-11-17,"[I693, E117, I10, H251, H544]",I693,"[E117, I10, H251, H544]",[],64,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-09-12,2022-11-17
111011,9,7522949,c26697771889db270b4708dbe98f68667773e98f09e11409b8619a6da70162a4,None,"1. T2DM\r\nFBS 163 HbA1c 7.3 (initial 13-14) --> 9.4 (21/4/65) \r\nทานข้าวเยอะ \r\ncomplications: ischemic stroke 2555 now Lt weak gr.0, bedbound, DR loss f/u\r\nincrease mixtard\r\n2. DLP LDL 102--> 114 , TG 189\r\n simvas to 40 mg/d\r\n3. CKD stage 2 stable,l UPCr >300",2022-11-17,"[I698, E119, E785, I10]",I698,"[E119, E785, I10]",[],65,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-30,2022-11-17
111012,15,7563561,b6cd79f852b3c34d8ff363efcf9351a7d4f73890c30237cc144c873b94dffef3,None,None,2022-11-17,"[I64, E876]",I64,[E876],[],56,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-16,2022-11-17
111013,13,7515954,d8f7c093acbdf5e05494de602301fe77e22e87cdbb30469a7cbbeb69b0aa5228,None,1.DM\r\nFBS 132 \r\n2. DLP\r\n3. HT \r\nBP 128/50\r\nannual lab 2/12/64 \r\nนัด 3 month +FBS+HbA1c\r\n\r\n18/11/65 รับยาไม่พบแพทย์ (วันหยุดพิเศษ),2022-11-18,"[I698, E116, E785, I10, M170]",I698,"[E116, E785, 

In [89]:
# data['min_y2'] = data['min_y'].dt.date
# data['max_y2'] = data['max_y'].dt.date

In [90]:
# def f(x):
#     return x.year
# data['min_y2'] = data['min_y2'].apply(f)
# data['max_y2'] = data['max_y2'].apply(f)

In [91]:
# def f(x):
#     return x.year
# data['max_y2'] = data['max_y2'].apply(f)

In [92]:
# ทำให้ col min,max เป็น date เก็บไว้ใน col min_2,max_2
data['min_2'] = data['min'].dt.date
data['max_2'] = data['max'].dt.date

#ทำให้ col min_2 , max_2 เป็น year
def d(x):
    return x.year
data['min_2'] = data['min_2'].apply(d)
data['max_2'] = data['max_2'].apply(d)

In [93]:
data

,visits,rowID,cid,vital_cc,vital_pe,visitDateTime,icd10_list,diagType_1,diagType_2,diagType_3,patientAge,patientSexName,smoking,narcotic,exercise,alcohol,vital_bw,vital_waist,vital_height,vital_bmi,vital_hr,vital_pulse,vital_temperature,vital_rr,vital_bps,vital_bpd,BUN,Cholesterol,Creatinine,FBS,LDL-Cholesterol,Triglyceride,Sodium,Potassium,Chloride,CO2,HCT,WBC,RBC,PLT,Neutrophil,Lymphocytes,Monocytes,Eosinophil,Basophil,MCV,MCH,MCHC,min,max,min_2,max_2
0,1,7629,4a1d93a977f66de044778ad54463bc516908785fd67ebe6e6299803733bdc623,None,None,2013-08-02,"[I639, E119, I10, E789]",I639,"[E119, I10, E789]",[],57,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-08-02,2021-10-04,2013,2021
1,1,94691,ee9c51648123950c3110a83355c6d2fdeb15b4a7b0995bad5f438fc9147a918f,None,None,2014-02-13,"[I635, E876, E789, E162, J029]",I635,"[E876, E789, E162, J029]",[],83,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-02-13,2022-10-31,2014,2022
2,1,18029,6b22580b026e54c109844361fddb1492cc0c16aa96cce6b09a45b1776eed5399,None,None,2014-03-24,"[I633, G819, E119, I10, E785, M1009]",I633,"[G819, E119, I10, E785, M1009]",[],60,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-03-24,2022-10-04,2014,2022
3,1,1572858,5379c7d14c13bd043ef5476ff887310345280b79578c0be5d618fcbe29ae77ad,None,None,2015-01-09,[I694],I694,[],[],63,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-09,2015-01-09,2015,2015
4,1,295810,eedebeecd8ed3c0dcf87872fa622268cff199fcfbf40c5707b691278f2df9ded,None,None,2015-01-19,"[I694, E119, I10]",I694,"[E119, I10]",[],81,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-19,2018-06-01,2015,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111010,3,7522927,8b790826af2f57d43f2011a05cfb2975a13683f47bfcbf572fadff0e8a2516d5,None,"Hx of DM, HT, Old CVA about 10 yr ago, add NPH 10 u OD for 3 yr. \r\n- ติด APEC เลื่อนนัด",2022-11-17,"[I693, E117, I10, H251, H544]",I693,"[E117, I10, H251, H544]",[],64,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-09-12,2022-11-17,2022,2022
111011,9,7522949,c26697771889db270b4708dbe98f68667773e98f09e11409b8619a6da70162a4,None,"1. T2DM\r\nFBS 163 HbA1c 7.3 (initial 13-14) --> 9.4 (21/4/65) \r\nทานข้าวเยอะ \r\ncomplications: ischemic stroke 2555 now Lt weak gr.0, bedbound, DR loss f/u\r\nincrease mixtard\r\n2. DLP LDL 102--> 114 , TG 189\r\n simvas to 40 mg/d\r\n3. CKD stage 2 stable,l UPCr >300",2022-11-17,"[I698, E119, E785, I10]",I698,"[E119, E785, I10]",[],65,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-30,2022-11-17,2020,2022
111012,15,7563561,b6cd79f852b3c34d8ff363efcf9351a7d4f73890c30237cc144c873b94dffef3,None,None,2022-11-17,"[I64, E876]",I64,[E876],[],56,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-16,2022-11-17,2021,2022
111013,13,7515954,d8f7c093acbdf5e05494de602301fe77e22e87cdbb30469a7cbbeb69b0aa5228,None,1.DM\r\nFBS 132 \r\n2. DLP\r\n3. HT \r\nBP 128/50\r\nannual lab 2/12/64 \r\nนัด 3 month +FBS+HbA1c\r\n\r\n18

In [94]:
# สร้าง col year 
data['year'] = data['max_2'] - data['min_2']
data

,visits,rowID,cid,vital_cc,vital_pe,visitDateTime,icd10_list,diagType_1,diagType_2,diagType_3,patientAge,patientSexName,smoking,narcotic,exercise,alcohol,vital_bw,vital_waist,vital_height,vital_bmi,vital_hr,vital_pulse,vital_temperature,vital_rr,vital_bps,vital_bpd,BUN,Cholesterol,Creatinine,FBS,LDL-Cholesterol,Triglyceride,Sodium,Potassium,Chloride,CO2,HCT,WBC,RBC,PLT,Neutrophil,Lymphocytes,Monocytes,Eosinophil,Basophil,MCV,MCH,MCHC,min,max,min_2,max_2,year
0,1,7629,4a1d93a977f66de044778ad54463bc516908785fd67ebe6e6299803733bdc623,None,None,2013-08-02,"[I639, E119, I10, E789]",I639,"[E119, I10, E789]",[],57,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-08-02,2021-10-04,2013,2021,8
1,1,94691,ee9c51648123950c3110a83355c6d2fdeb15b4a7b0995bad5f438fc9147a918f,None,None,2014-02-13,"[I635, E876, E789, E162, J029]",I635,"[E876, E789, E162, J029]",[],83,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-02-13,2022-10-31,2014,2022,8
2,1,18029,6b22580b026e54c109844361fddb1492cc0c16aa96cce6b09a45b1776eed5399,None,None,2014-03-24,"[I633, G819, E119, I10, E785, M1009]",I633,"[G819, E119, I10, E785, M1009]",[],60,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-03-24,2022-10-04,2014,2022,8
3,1,1572858,5379c7d14c13bd043ef5476ff887310345280b79578c0be5d618fcbe29ae77ad,None,None,2015-01-09,[I694],I694,[],[],63,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-09,2015-01-09,2015,2015,0
4,1,295810,eedebeecd8ed3c0dcf87872fa622268cff199fcfbf40c5707b691278f2df9ded,None,None,2015-01-19,"[I694, E119, I10]",I694,"[E119, I10]",[],81,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-19,2018-06-01,2015,2018,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111010,3,7522927,8b790826af2f57d43f2011a05cfb2975a13683f47bfcbf572fadff0e8a2516d5,None,"Hx of DM, HT, Old CVA about 10 yr ago, add NPH 10 u OD for 3 yr. \r\n- ติด APEC เลื่อนนัด",2022-11-17,"[I693, E117, I10, H251, H544]",I693,"[E117, I10, H251, H544]",[],64,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-09-12,2022-11-17,2022,2022,0
111011,9,7522949,c26697771889db270b4708dbe98f68667773e98f09e11409b8619a6da70162a4,None,"1. T2DM\r\nFBS 163 HbA1c 7.3 (initial 13-14) --> 9.4 (21/4/65) \r\nทานข้าวเยอะ \r\ncomplications: ischemic stroke 2555 now Lt weak gr.0, bedbound, DR loss f/u\r\nincrease mixtard\r\n2. DLP LDL 102--> 114 , TG 189\r\n simvas to 40 mg/d\r\n3. CKD stage 2 stable,l UPCr >300",2022-11-17,"[I698, E119, E785, I10]",I698,"[E119, E785, I10]",[],65,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-30,2022-11-17,2020,2022,2
111012,15,7563561,b6cd79f852b3c34d8ff363efcf9351a7d4f73890c30237cc144c873b94dffef3,None,None,2022-11-17,"[I64, E876]",I64,[E876],[],56,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-16,2022-11-17,2021,2022,1
111013,13,7515954,d8f7c093acbdf5e05494de602301fe77e22e87cdbb30469a7cbbeb69b0aa5228,None,1.DM\r\nFBS 132 \r\n2. DLP\r\n3. HT \r\nBP 128/50\r\nannual lab 2/12/64 \r\nนัด 3 m

In [95]:
data[data['cid'] == '4a1d93a977f66de044778ad54463bc516908785fd67ebe6e6299803733bdc623']

,visits,rowID,cid,vital_cc,vital_pe,visitDateTime,icd10_list,diagType_1,diagType_2,diagType_3,patientAge,patientSexName,smoking,narcotic,exercise,alcohol,vital_bw,vital_waist,vital_height,vital_bmi,vital_hr,vital_pulse,vital_temperature,vital_rr,vital_bps,vital_bpd,BUN,Cholesterol,Creatinine,FBS,LDL-Cholesterol,Triglyceride,Sodium,Potassium,Chloride,CO2,HCT,WBC,RBC,PLT,Neutrophil,Lymphocytes,Monocytes,Eosinophil,Basophil,MCV,MCH,MCHC,min,max,min_2,max_2,year
0,1,7629,4a1d93a977f66de044778ad54463bc516908785fd67ebe6e6299803733bdc623,None,None,2013-08-02,"[I639, E119, I10, E789]",I639,"[E119, I10, E789]",[],57,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-08-02,2021-10-04,2013,2021,8
30619,2,4078171,4a1d93a977f66de044778ad54463bc516908785fd67ebe6e6299803733bdc623,แขนขาข้างซ้ายอ่อนแรง 3 ชม. ก่อนมา,None,2020-06-05,[I64],I64,[],[],64,1.0,None,None,None,None,60.0,70.0,165.0,22.039,None,73.0,36.8,18.0,136.0,73.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-08-02,2021-10-04,2013,2021,8
30659,3,7617,4a1d93a977f66de044778ad54463bc516908785fd67ebe6e6299803733bdc623,None,None,2020-06-05,"[I639, I10, E119, N183, I694, E789]",I639,"[I10, E119, N183, I694, E789]",[],64,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,25.00,135.00,2.52,193.00,66.00,134.00,138.00,4.20,103.00,20.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-08-02,2021-10-04,2013,2021,8
30660,4,4139948,4a1d93a977f66de044778ad54463bc516908785fd67ebe6e6299803733bdc623,ปากเบี้ยว ปวดศีรษะ ข้างขวา อ่อนแรงแขนขาซ้าย 10.30 น.,None,2020-06-05,[I64],I64,[],[],64,1.0,None,None,None,None,None,None,None,0.0,None,73.0,36.4,20.0,156.0,81.0,NaN,NaN,NaN,220.00,NaN,NaN,NaN,NaN,NaN,NaN,41.00,NaN,4.40,NaN,79.00,14.00,4.00,2.00,1.00,93.00,31.00,34.00,2013-08-02,2021-10-04,2013,2021,8
30759,5,4078129,4a1d93a977f66de044778ad54463bc516908785fd67ebe6e6299803733bdc623,รับกลับจากพสธ.รักษาต่อเนื่อง,None,2020-06-08,[I64],I64,[],[],64,1.0,None,None,None,None,60.0,70.0,165.0,22.039,None,88.0,36.8,18.0,178.0,88.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-08-02,2021-10-04,2013,2021,8
57067,6,4078231,4a1d93a977f66de044778ad54463bc516908785fd67ebe6e6299803733bdc623,case stroke พอเดินได้ ญาติมารับยาแทน,None,2021-04-25,[I694],I694,[],[],65,1.0,None,None,None,None,60.0,70.0,165.0,22.039,None,68.0,36.5,16.0,130.0,80.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-08-02,2021-10-04,2013,2021,8
69686,7,2068268,4a1d93a977f66de044778ad54463bc516908785fd67ebe6e6299803733bdc623,case stroke พอเดินได้ ญาติมารับยาแทน,None,2021-10-04,"[I694, I10, E134, N184]",I694,"[I10, E134, N184]",[],65,1.0,None,None,None,None,60.0,70.0,165.0,22.039,None,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-08-02,2021-10-04,2013,2021,8


In [96]:
# เช็คว่าส่วนมากผู้ป่วยมาทำอะไร
data[(data['cid'] == 'b2f2bdc360326c5753950eda4644c404b71b6ad3ae1e186550dd4b610c0a0122')][['visits','visitDateTime','vital_cc','year','Cholesterol']].sort_values(by='visits', ascending=True)

,visits,visitDateTime,vital_cc,year,Cholesterol
132,1,2017-01-03,ติดตามการเยี่ยมบ้านผู้ป่วยโรคเรื้อรัง,4,NaN
163,2,2017-01-05,ชา และแขนขาขวาอ่อนแรง,4,NaN
217,3,2017-01-10,ติดตามการเยี่ยมบ้านผู้ป่วยโรคเรื้อรัง,4,NaN
248,4,2017-01-11,ชา และแขนขาขวาอ่อนแรง,4,NaN
284,5,2017-01-16,ติดตามการเยี่ยมบ้านผู้ปวยโรคเรื้อรัง,4,NaN
...,...,...,...,...,...
15538,157,2019-08-08,ชา และแขนขาขวาอ่อนแรง เดินก้าวยังขัด,4,NaN
16037,158,2019-08-29,ชา และแขนขาขวาอ่อนแรง เดินก้าวยังขัด,4,NaN
16207,159,2019-09-05,ปวดเข่าสองข้าง เป็นมา 1 เดือน,4,NaN
57331,160,2021-04-27,อ่อนแรงแขนขาซีกขวา พูดไม่ชัด,4,NaN


In [97]:
# df2 = data.explode('diagType_2').reset_index(drop=True)
# df2

In [98]:
# # เลือกเอาเฉพาะคนที่มี visits = 1 because visits = 1 is ผู้ป่วยมาครั้งเเรก
# data1 = data[(data['visits'] == 1)].sort_values(by='visitDateTime', ascending=True)
# data1

In [99]:
data

,visits,rowID,cid,vital_cc,vital_pe,visitDateTime,icd10_list,diagType_1,diagType_2,diagType_3,patientAge,patientSexName,smoking,narcotic,exercise,alcohol,vital_bw,vital_waist,vital_height,vital_bmi,vital_hr,vital_pulse,vital_temperature,vital_rr,vital_bps,vital_bpd,BUN,Cholesterol,Creatinine,FBS,LDL-Cholesterol,Triglyceride,Sodium,Potassium,Chloride,CO2,HCT,WBC,RBC,PLT,Neutrophil,Lymphocytes,Monocytes,Eosinophil,Basophil,MCV,MCH,MCHC,min,max,min_2,max_2,year
0,1,7629,4a1d93a977f66de044778ad54463bc516908785fd67ebe6e6299803733bdc623,None,None,2013-08-02,"[I639, E119, I10, E789]",I639,"[E119, I10, E789]",[],57,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-08-02,2021-10-04,2013,2021,8
1,1,94691,ee9c51648123950c3110a83355c6d2fdeb15b4a7b0995bad5f438fc9147a918f,None,None,2014-02-13,"[I635, E876, E789, E162, J029]",I635,"[E876, E789, E162, J029]",[],83,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-02-13,2022-10-31,2014,2022,8
2,1,18029,6b22580b026e54c109844361fddb1492cc0c16aa96cce6b09a45b1776eed5399,None,None,2014-03-24,"[I633, G819, E119, I10, E785, M1009]",I633,"[G819, E119, I10, E785, M1009]",[],60,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-03-24,2022-10-04,2014,2022,8
3,1,1572858,5379c7d14c13bd043ef5476ff887310345280b79578c0be5d618fcbe29ae77ad,None,None,2015-01-09,[I694],I694,[],[],63,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-09,2015-01-09,2015,2015,0
4,1,295810,eedebeecd8ed3c0dcf87872fa622268cff199fcfbf40c5707b691278f2df9ded,None,None,2015-01-19,"[I694, E119, I10]",I694,"[E119, I10]",[],81,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-19,2018-06-01,2015,2018,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111010,3,7522927,8b790826af2f57d43f2011a05cfb2975a13683f47bfcbf572fadff0e8a2516d5,None,"Hx of DM, HT, Old CVA about 10 yr ago, add NPH 10 u OD for 3 yr. \r\n- ติด APEC เลื่อนนัด",2022-11-17,"[I693, E117, I10, H251, H544]",I693,"[E117, I10, H251, H544]",[],64,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-09-12,2022-11-17,2022,2022,0
111011,9,7522949,c26697771889db270b4708dbe98f68667773e98f09e11409b8619a6da70162a4,None,"1. T2DM\r\nFBS 163 HbA1c 7.3 (initial 13-14) --> 9.4 (21/4/65) \r\nทานข้าวเยอะ \r\ncomplications: ischemic stroke 2555 now Lt weak gr.0, bedbound, DR loss f/u\r\nincrease mixtard\r\n2. DLP LDL 102--> 114 , TG 189\r\n simvas to 40 mg/d\r\n3. CKD stage 2 stable,l UPCr >300",2022-11-17,"[I698, E119, E785, I10]",I698,"[E119, E785, I10]",[],65,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-30,2022-11-17,2020,2022,2
111012,15,7563561,b6cd79f852b3c34d8ff363efcf9351a7d4f73890c30237cc144c873b94dffef3,None,None,2022-11-17,"[I64, E876]",I64,[E876],[],56,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-16,2022-11-17,2021,2022,1
111013,13,7515954,d8f7c093acbdf5e05494de602301fe77e22e87cdbb30469a7cbbeb69b0aa5228,None,1.DM\r\nFBS 132 \r\n2. DLP\r\n3. HT \r\nBP 128/50\r\nannual lab 2/12/64 \r\nนัด 3 m

## โรคร่วม

In [100]:
# data1['diagType_1'] = data1['diagType_1'].apply(lambda x: f(x))
# data1['diagType_2'] = data1['diagType_2'].apply(lambda x: f(x))
# data1['diagType_3'] = data1['diagType_3'].apply(lambda x: f(x))

In [101]:
data

,visits,rowID,cid,vital_cc,vital_pe,visitDateTime,icd10_list,diagType_1,diagType_2,diagType_3,patientAge,patientSexName,smoking,narcotic,exercise,alcohol,vital_bw,vital_waist,vital_height,vital_bmi,vital_hr,vital_pulse,vital_temperature,vital_rr,vital_bps,vital_bpd,BUN,Cholesterol,Creatinine,FBS,LDL-Cholesterol,Triglyceride,Sodium,Potassium,Chloride,CO2,HCT,WBC,RBC,PLT,Neutrophil,Lymphocytes,Monocytes,Eosinophil,Basophil,MCV,MCH,MCHC,min,max,min_2,max_2,year
0,1,7629,4a1d93a977f66de044778ad54463bc516908785fd67ebe6e6299803733bdc623,None,None,2013-08-02,"[I639, E119, I10, E789]",I639,"[E119, I10, E789]",[],57,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-08-02,2021-10-04,2013,2021,8
1,1,94691,ee9c51648123950c3110a83355c6d2fdeb15b4a7b0995bad5f438fc9147a918f,None,None,2014-02-13,"[I635, E876, E789, E162, J029]",I635,"[E876, E789, E162, J029]",[],83,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-02-13,2022-10-31,2014,2022,8
2,1,18029,6b22580b026e54c109844361fddb1492cc0c16aa96cce6b09a45b1776eed5399,None,None,2014-03-24,"[I633, G819, E119, I10, E785, M1009]",I633,"[G819, E119, I10, E785, M1009]",[],60,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-03-24,2022-10-04,2014,2022,8
3,1,1572858,5379c7d14c13bd043ef5476ff887310345280b79578c0be5d618fcbe29ae77ad,None,None,2015-01-09,[I694],I694,[],[],63,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-09,2015-01-09,2015,2015,0
4,1,295810,eedebeecd8ed3c0dcf87872fa622268cff199fcfbf40c5707b691278f2df9ded,None,None,2015-01-19,"[I694, E119, I10]",I694,"[E119, I10]",[],81,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-19,2018-06-01,2015,2018,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111010,3,7522927,8b790826af2f57d43f2011a05cfb2975a13683f47bfcbf572fadff0e8a2516d5,None,"Hx of DM, HT, Old CVA about 10 yr ago, add NPH 10 u OD for 3 yr. \r\n- ติด APEC เลื่อนนัด",2022-11-17,"[I693, E117, I10, H251, H544]",I693,"[E117, I10, H251, H544]",[],64,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-09-12,2022-11-17,2022,2022,0
111011,9,7522949,c26697771889db270b4708dbe98f68667773e98f09e11409b8619a6da70162a4,None,"1. T2DM\r\nFBS 163 HbA1c 7.3 (initial 13-14) --> 9.4 (21/4/65) \r\nทานข้าวเยอะ \r\ncomplications: ischemic stroke 2555 now Lt weak gr.0, bedbound, DR loss f/u\r\nincrease mixtard\r\n2. DLP LDL 102--> 114 , TG 189\r\n simvas to 40 mg/d\r\n3. CKD stage 2 stable,l UPCr >300",2022-11-17,"[I698, E119, E785, I10]",I698,"[E119, E785, I10]",[],65,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-30,2022-11-17,2020,2022,2
111012,15,7563561,b6cd79f852b3c34d8ff363efcf9351a7d4f73890c30237cc144c873b94dffef3,None,None,2022-11-17,"[I64, E876]",I64,[E876],[],56,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-16,2022-11-17,2021,2022,1
111013,13,7515954,d8f7c093acbdf5e05494de602301fe77e22e87cdbb30469a7cbbeb69b0aa5228,None,1.DM\r\nFBS 132 \r\n2. DLP\r\n3. HT \r\nBP 128/50\r\nannual lab 2/12/64 \r\nนัด 3 m

In [102]:
# .explode() : แตกข้อมูลเเต่ละตัวใน list ออกมาเป็นแถว เพราะในเเต่ละ diagType มีข้อมูลหลายตัวใน list
# value_counts() : นับค่า icd10 ของเเต่ละ diagType
# .reset_index : Index เก่าจะถูกเพิ่มเป็นคอลัมน์ เเละเพิ่ม index ขึ้นมาใหม่
df_count_1 = data['diagType_1'].explode().value_counts().reset_index(name='counts') # name='counts' บอกชื่อ col
df_count_2 = data['diagType_2'].explode().value_counts().reset_index(name='counts')
df_count_3 = data['diagType_3'].explode().value_counts().reset_index(name='counts')

In [103]:
# top 10 diagType_2

df_count_2.head(10)

# I10 Hypertension / ค.ดันโลหิตสูง
# E785 Hyperlipidaemia / ไขมันในเลือดสูง 
# E119 dm / เบาหวานชนิดที่ 2
# E789 Disorder of lipoprotein metabolism / ความผิดปกติของการเผาผลาญไลโปโปรตีน
# G819 Hemiplegia / อัมพาตครึ่งซีก
# N183 Chronic kidney disease, stage 3 / ไตเรื้อรังระยะ 3
# N182 Chronic kidney disease, stage 2 / ไตเรื้อรังระยะ 2
# E780 Pure hypercholesterolemia / ไขมันในเลือดสูง
# I48  Atrial fibrillation and flutter / ภาวะหัวใจเต้นผิดจังหวะ
# E876 Hypokalemia / ภาวะโพแทสเซียมสูง

,index,counts
0,I10,38976
1,E785,15293
2,E119,12483
3,E789,10489
4,G819,4232
5,N183,3213
6,N182,2492
7,E780,2201
8,I48,1941
9,E876,1733


In [104]:
# Create feature columns - for icd10 codes
# เช็คว่ามี str นั้น ๆ อยู่ใน list ไหม ถ้าใช่ก็ 1 ไม่ใช่ก็ 0
def check_substr_list(x_list, substr):
    for i in x_list:
        if substr in i:
            return 1
    return 0
    
def check_multiple_substr(x_list, substr_list):
    for i in x_list:
        for j in substr_list:
            if j in i:
                return 1            
    return 0

In [105]:
# Extarct ICD10 associated with DM

Hypertension = ['I10', 'I11', 'I12', 'I13', 'I14', 'I16'] # ความดันโลหิตสูง

# hd = ['I21','I210','I211','I212','I213','I214','I219','I22','I220','I221','I228','I229','I23','I231','I232','I233','I234','I235','I236','I238','I24'
#       ,'I240','I241','I248','I249','I25','I250','I252','I253','I254','I255','I259','I258','I256'] # โรคหัวใจ

Hyperlipidaemia = ['E780','E785','E789'] # Hyperlipidaemia / ไขมันในเลือดสูง 

Diabetes_Mellitus = ['E11','E110','E111','E112','E113','E114','E115','E116','E117','E118','E119'] # diabetes type2


Hemiplegia = ['G819'] # Hemiplegia / อัมพาตครึ่งซีก

Chronic_kidney = ['N182','N183'] # Chronic kidney disease / ไตเรื้อรัง

Atrial_fibrillation = ['I48']# Atrial fibrillation and flutter / ภาวะหัวใจเต้นผิดจังหวะ

Hypokalemia = ['E876'] # Hypokalemia / ภาวะโพแทสเซียมสูง


data["icd10_list"] = data["icd10_list"].apply(lambda x: f(x))

data["Hypertension"] = data["icd10_list"].apply(check_multiple_substr, substr_list = Hypertension)
data["Hyperlipidaemia"] = data["icd10_list"].apply(check_multiple_substr, substr_list = Hyperlipidaemia)
data["Diabetes_Mellitus"] = data["icd10_list"].apply(check_multiple_substr, substr_list = Diabetes_Mellitus)
data["Hemiplegia"] = data["icd10_list"].apply(check_multiple_substr, substr_list = Hemiplegia)
data["Chronic_kidney"] = data["icd10_list"].apply(check_multiple_substr, substr_list = Chronic_kidney)
data["Atrial_fibrillation"] = data["icd10_list"].apply(check_multiple_substr, substr_list = Atrial_fibrillation)
data["Hypokalemia"] = data["icd10_list"].apply(check_multiple_substr, substr_list = Hypokalemia)

# data["pcos"] = data["icd10_list"].apply(check_substr_list, substr = 'E282')
# data["hd"] = data["icd10_list"].apply(check_multiple_substr, substr_list = hd)

In [106]:
# สร้าง col stroke ขึ้นมาโดยมีเงื่อไขว่า ถ้า 'icd10_list' มีตัว I63 อยู่ในนั้น ให้มีค่าเป็น 1 ถ้าไม่มีให้เป็น 0
data['stroke'] = np.where((data['icd10_list'].str.contains("I6")),1,0)
data

,visits,rowID,cid,vital_cc,vital_pe,visitDateTime,icd10_list,diagType_1,diagType_2,diagType_3,patientAge,patientSexName,smoking,narcotic,exercise,alcohol,vital_bw,vital_waist,vital_height,vital_bmi,vital_hr,vital_pulse,vital_temperature,vital_rr,vital_bps,vital_bpd,BUN,Cholesterol,Creatinine,FBS,LDL-Cholesterol,Triglyceride,Sodium,Potassium,Chloride,CO2,HCT,WBC,RBC,PLT,Neutrophil,Lymphocytes,Monocytes,Eosinophil,Basophil,MCV,MCH,MCHC,min,max,min_2,max_2,year,Hypertension,Hyperlipidaemia,Diabetes_Mellitus,Hemiplegia,Chronic_kidney,Atrial_fibrillation,Hypokalemia,stroke
0,1,7629,4a1d93a977f66de044778ad54463bc516908785fd67ebe6e6299803733bdc623,None,None,2013-08-02,[I639E119I10E789],I639,"[E119, I10, E789]",[],57,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-08-02,2021-10-04,2013,2021,8,1,1,1,0,0,0,0,1
1,1,94691,ee9c51648123950c3110a83355c6d2fdeb15b4a7b0995bad5f438fc9147a918f,None,None,2014-02-13,[I635E876E789E162J029],I635,"[E876, E789, E162, J029]",[],83,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-02-13,2022-10-31,2014,2022,8,0,1,0,0,0,0,1,1
2,1,18029,6b22580b026e54c109844361fddb1492cc0c16aa96cce6b09a45b1776eed5399,None,None,2014-03-24,[I633G819E119I10E785M1009],I633,"[G819, E119, I10, E785, M1009]",[],60,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-03-24,2022-10-04,2014,2022,8,1,1,1,1,0,0,0,1
3,1,1572858,5379c7d14c13bd043ef5476ff887310345280b79578c0be5d618fcbe29ae77ad,None,None,2015-01-09,[I694],I694,[],[],63,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-09,2015-01-09,2015,2015,0,0,0,0,0,0,0,0,1
4,1,295810,eedebeecd8ed3c0dcf87872fa622268cff199fcfbf40c5707b691278f2df9ded,None,None,2015-01-19,[I694E119I10],I694,"[E119, I10]",[],81,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-19,2018-06-01,2015,2018,3,1,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111010,3,7522927,8b790826af2f57d43f2011a05cfb2975a13683f47bfcbf572fadff0e8a2516d5,None,"Hx of DM, HT, Old CVA about 10 yr ago, add NPH 10 u OD for 3 yr. \r\n- ติด APEC เลื่อนนัด",2022-11-17,[I693E117I10H251H544],I693,"[E117, I10, H251, H544]",[],64,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-09-12,2022-11-17,2022,2022,0,1,0,1,0,0,0,0,1
111011,9,7522949,c26697771889db270b4708dbe98f68667773e98f09e11409b8619a6da70162a4,None,"1. T2DM\r\nFBS 163 HbA1c 7.3 (initial 13-14) --> 9.4 (21/4/65) \r\nทานข้าวเยอะ \r\ncomplications: ischemic stroke 2555 now Lt weak gr.0, bedbound, DR loss f/u\r\nincrease mixtard\r\n2. DLP LDL 102--> 114 , TG 189\r\n simvas to 40 mg/d\r\n3. CKD stage 2 stable,l UPCr >300",2022-11-17,[I698E119E785I10],I698,"[E119, E785, I10]",[],65,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-30,2022-11-17,2020,2022,2,1,1,1,0,0,0,0,1
111012,15,7563561,b6cd79f852b3c34d8ff363efcf9351a7d4f73890c30237cc144c873b94dffef3,None,None,2022-11-17,[I64E876],I64,[E876],[],56,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08

In [107]:
data.drop(columns= ["icd10_list",'vital_cc','vital_pe','diagType_1','diagType_2'], inplace = True)

In [108]:
data.drop(columns= ['diagType_3'], inplace = True)

In [109]:
data.to_csv('Clean_1.csv', index= False)

## ทำ feature engineering ดูที่พวก vital_cc 

In [ ]:
# def extract_condition(row, keywords_0, keywords_1):
#     for col in ["vital_cc", "vital_pe", "vital_hpi"]:
#         text = row[col]
#         if text:
#             text = text.lower()
#             match_0 = re.search("|".join(keywords_0), text)
#             match_1 = re.search("|".join(keywords_1), text)
#             if match_0:
#                 return "Yes"
#             elif match_1:
#                 return "No"
#     return None

In [ ]:
# # numbness
# keywords_0 = ["ไม่ชา","ไม่มีชา"]
# keywords_1 = ["ชาแขนซ้าย","มีชา","ชาซีกขวา","ชาตามตัว","ชาปลายมือปลายเท้า","ชาปลายเท้า",
#               "มีอาการชา","เท้าชา","ขาชา","มือ.*ชา","ขาชา","ชาขา","ชาเท้า","ชาที่","รู้สึกชา"
#               ,"ชาแขน","แขนข้างขวาชา","ชาปลายนิ้ว","ชามือ","ชา.*มือ","ชาฝ่าเท้า"]
# IHD["numbness"] = IHD.apply(lambda x: extract_condition(x, keywords_0, keywords_1), axis=1)